# Marktstammdaten 

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import sqlite3
import matplotlib.pyplot as plt
import geopandas as gpd
import datetime, time
from decimal import Decimal

## Marktstammdaten einlesen & säubern

In [2]:
# Marktstammdaten-Tabelle einlesen
pv_raw_clean = pd.read_pickle('marktstammdaten.pkl')
pv_raw_clean.head(5)

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,...,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,AGS-5,Inbetriebnahmejahr
0,2020-02-20T16:28:35.2508122,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01,...,Kein Stromspeicher vorhanden,32,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,20 - 40 Grad,Haushalt,05515,2007
1,2021-07-15T07:54:37.8637095,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,...,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,08136,2013
2,2019-05-28T08:26:38.0316692,1,ABR983885950021,Brandenburg,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,...,Stromspeicher vorhanden - kein gemeinsamer Wec...,22,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,12063,2016
3,2020-12-07T08:19:07.9860793,1,ABR980052996196,Bayern,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,...,Stromspeicher vorhanden - gemeinsamer Wechselr...,27,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, sonstige",1.0,Süd-West,20 - 40 Grad,Haushalt,09375,2016
4,2021-01-25T11:18:30.0433603,1,ABR970373690446,Saarland,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,...,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,0.0,West,20 - 40 Grad,Haushalt,10044,2011


In [3]:
# pv_raw_clean.isnull().sum()

In [4]:
marktstd = pv_raw_clean.copy()
#marktstd.head()

Es fehlen Informationen zu 3 Gemeindeschlüsseln, da es seit 2011 Gebietsveränderungen gab:  
**16056:** Stadt Eisenach, ab 2021 16063    
**03152:** Göttingen, ab 2016 03159   
**03156:** Kreis Osterode am Harz, ab 2016 03159 (Göttingen)   

In [5]:
# betreffende Zeilen in der Marktstammdaten-Tabelle korrigieren
# 16056: Stadt Eisenach, ab 2021 16063
marktstd.loc[marktstd['AGS-5'] == '16056', 'AGS-5'] = '16063'
marktstd.loc[marktstd['AGS-8'] == '16056000', 'AGS-8'] = '16063000'

# 03152: Göttingen, ab 2016 03159
marktstd.loc[marktstd['AGS-5'] == '03152', 'AGS-5'] = '03159'
marktstd.loc[marktstd['AGS-8'] == '03159000', 'AGS-8'] = '03159000'

# 03156: Kreis Osterode am Harz, ab 2016 03159 (Göttingen)
marktstd.loc[marktstd['AGS-5'] == '03156', 'AGS-5'] = '03159'
marktstd.loc[marktstd['AGS-8'] == '03156000', 'AGS-8'] = '03159000'

Für drei Einträge wurde der Bundesland-Code 1416 (Ausschließliche Wirtschaftszone) angegeben.     
Tatsächlich liegen die Orte jedoch in anderen Bundesländern:  
>  
ABR995244487963	1416	Rendsburg-Eckernförde	Hohenwestedt	24594	Hohenwestedt   
> Korrektur auf Schleswig-Holstein   
ABR979285762777	1416	Dithmarschen	Eddelak	25715	Eddelak	    
> Korrektur auf Schleswig-Holstein    
ABR918452306895	1416	Eichsfeld	Dingelstädt	37351	Kefferhausen     
> Korrektur auf Thüringen     

In [6]:
marktstd.loc[marktstd['AnlagenbetreiberMastrNummer'] == 'ABR995244487963', 'Bundesland'] = 'Schleswig-Holstein'
marktstd.loc[marktstd['AnlagenbetreiberMastrNummer'] == 'ABR979285762777', 'Bundesland'] = 'Schleswig-Holstein'
marktstd.loc[marktstd['AnlagenbetreiberMastrNummer'] == 'ABR918452306895', 'Bundesland'] = 'Thüringen'

In [7]:
# Datumsangaben formatieren
marktstd['DatumLetzteAktualisierung'] = pd.to_datetime(marktstd['DatumLetzteAktualisierung'])
marktstd['Registrierungsdatum'] = pd.to_datetime(marktstd['Registrierungsdatum'])
marktstd['Inbetriebnahmedatum'] = pd.to_datetime(marktstd['Inbetriebnahmedatum'])
marktstd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3173010 entries, 0 to 3173009
Data columns (total 26 columns):
 #   Column                                    Dtype         
---  ------                                    -----         
 0   DatumLetzteAktualisierung                 datetime64[ns]
 1   NetzbetreiberpruefungStatus               int64         
 2   AnlagenbetreiberMastrNummer               object        
 3   Bundesland                                object        
 4   Landkreis                                 object        
 5   Gemeinde                                  object        
 6   AGS-8                                     object        
 7   Postleitzahl                              object        
 8   Ort                                       object        
 9   Registrierungsdatum                       datetime64[ns]
 10  Inbetriebnahmedatum                       datetime64[ns]
 11  EinheitBetriebsstatus                     object        
 12  Bruttoleistung

In [8]:
#marktstd.isnull().sum()

In [9]:
# Bereinigten Datensatz als Pickle speichern
marktstd = marktstd.copy()
marktstd.to_pickle('marktstammdaten_ags_plz.pkl')

# Sozio-ökonomische Daten

## Fläche, Bevölkerung und Bevölkerungsdichte
Stand: 31.12.2021   
Quelle: https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise  

In [10]:
# Datensatz einlesen
dichte_raw = pd.read_excel("../input/Data/Bevoelkerungsdichte.xlsx", sheet_name="Kreisfreie Städte u. Landkreise", index_col=0, )
dichte_raw.drop(dichte_raw.tail(11).index,
        inplace = True)
dichte_raw = dichte_raw.rename(columns={'Unnamed: 6': 'Bevölkerung pro km2', 'Bevölkerung2)': 'Bevölkerung gesamt', 
                                        'Fläche \nin km2': 'Fläche in km2'})
# dichte_raw

### Data Cleaning

In [11]:
dichte_raw = dichte_raw.reset_index()
dichte_raw['Gemeindeschlüssel'] = dichte_raw['Gemeindeschlüssel'].astype(str)
dichte_raw = dichte_raw.rename(columns={'Gemeindeschlüssel': 'AGS-5'})
dichte_raw

,AGS-5,Bezeichnung,Name,NUTS3,Fläche km2,Bevölkerung,Bevölkerung pro km2
0,01001,Kreisfreie Stadt,"Flensburg, Stadt",DEF01,56.73,91113.0,1606.0
1,01002,Kreisfreie Stadt,"Kiel, Landeshauptstadt",DEF02,118.65,246243.0,2075.0
2,01003,Kreisfreie Stadt,"Lübeck, Hansestadt",DEF03,214.19,216277.0,1010.0
3,01004,Kreisfreie Stadt,"Neumünster, Stadt",DEF04,71.66,79496.0,1109.0
4,01051,Kreis,Dithmarschen,DEF05,1428.17,133969.0,94.0
...,...,...,...,...,...,...,...
395,16073,Landkreis,Saalfeld-Rudolstadt,DEG0I,1008.78,100969.0,100.0
396,16074,Landkreis,Saale-Holzland-Kreis,DEG0J,815.24,82513.0,101.0
397,16075,Landkreis,Saale-Orla-Kreis,DEG0K,1151.30,79030.0,69.0
398,16076,Landkreis,Greiz,DEG0L,845.98,96102.0,114.0


In [12]:
# Ist der Gemeindeschlüssel unique?
count = dichte_raw['AGS-5'].value_counts()
not_unique_gs = count[count > 1]
not_unique_gs

Series([], Name: count, dtype: int64)

In [13]:
# nicht benötigte Spalten löschen
bev_dichte= dichte_raw.drop(['Bezeichnung', 'Name', 'NUTS3'], axis=1)
bev_dichte

,AGS-5,Fläche km2,Bevölkerung,Bevölkerung pro km2
0,01001,56.73,91113.0,1606.0
1,01002,118.65,246243.0,2075.0
2,01003,214.19,216277.0,1010.0
3,01004,71.66,79496.0,1109.0
4,01051,1428.17,133969.0,94.0
...,...,...,...,...
395,16073,1008.78,100969.0,100.0
396,16074,815.24,82513.0,101.0
397,16075,1151.30,79030.0,69.0
398,16076,845.98,96102.0,114.0


### Test Merge

In [14]:
# TEST merge mit Marktstammdaten
#pd.set_option('display.max_columns', 100)
#marktstd = marktstd.merge(bev_dichte, on="AGS-5", how="left")
#marktstd.head()

In [15]:
#marktstd.isna().sum()

## Entwicklung der Bevölkerungszahl
**zwischen 2016 und 2021 pro Jahr in %**   
Die gemittelte Entwicklung pro Jahr wird berechnet aus der Differenz der Bevölkerungsstände 2021   
und 2016 geteilt durch 5, bezogen auf den Bevölkerungsstand 2016. Stichtag ist jeweils der 31.12.   
Quelle: https://mis.bkg.bund.de/trefferanzeige?docuuid=0FF7C595-E012-4ED3-AD7E-F9D9F031E147


In [16]:
# Datensatz einlesen
columns = ["Gebietskennziffer", "name", "bev_entw"]
bev_entw_raw = pd.read_csv("../input/Deutschlandatlas/Deutschlandatlas_GEM1221.csv", usecols=columns, encoding="latin1", 
                              dtype={"Gebietskennziffer" : str}, sep=";", decimal=",", low_memory=False)

bev_entw_raw

,Gebietskennziffer,name,bev_entw
0,1001000,"Flensburg, Stadt",0.84
1,1002000,"Kiel, Landeshauptstadt",-0.10
2,1003000,"Lübeck, Hansestadt",-0.04
3,1004000,"Neumünster, Stadt",-0.05
4,1051001,Albersdorf,0.27
...,...,...,...
10989,16077044,Starkenberg,-0.76
10990,16077047,Thonhausen,-0.88
10991,16077048,Treben,-0.62
10992,16077049,Vollmershain,-0.57


### DataCleaning

In [17]:
bev_entw_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10994 entries, 0 to 10993
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gebietskennziffer  10994 non-null  object 
 1   name               10994 non-null  object 
 2   bev_entw           10994 non-null  float64
dtypes: float64(1), object(2)
memory usage: 257.8+ KB


In [18]:
duplikate = bev_entw_raw[bev_entw_raw['Gebietskennziffer'].duplicated()]
duplikate

,Gebietskennziffer,name,bev_entw


In [19]:
# Gebietskennziffer umwandeln in Gemeindeschlüssel AGS 8-stellig
bev_entw_raw['Gebietskennziffer'] = bev_entw_raw['Gebietskennziffer'].apply(lambda x: '0' + x if len(x) == 7 else x)
bev_entw_raw

,Gebietskennziffer,name,bev_entw
0,01001000,"Flensburg, Stadt",0.84
1,01002000,"Kiel, Landeshauptstadt",-0.10
2,01003000,"Lübeck, Hansestadt",-0.04
3,01004000,"Neumünster, Stadt",-0.05
4,01051001,Albersdorf,0.27
...,...,...,...
10989,16077044,Starkenberg,-0.76
10990,16077047,Thonhausen,-0.88
10991,16077048,Treben,-0.62
10992,16077049,Vollmershain,-0.57


In [20]:
bev_entw_raw = bev_entw_raw.rename(columns={'bev_entw': 'Bev_Entwicklung_%', 'name' : 'Ort',
                                           'Gebietskennziffer' : 'AGS-8'})
bev_entw_raw

,AGS-8,Ort,Bev_Entwicklung_%
0,01001000,"Flensburg, Stadt",0.84
1,01002000,"Kiel, Landeshauptstadt",-0.10
2,01003000,"Lübeck, Hansestadt",-0.04
3,01004000,"Neumünster, Stadt",-0.05
4,01051001,Albersdorf,0.27
...,...,...,...
10989,16077044,Starkenberg,-0.76
10990,16077047,Thonhausen,-0.88
10991,16077048,Treben,-0.62
10992,16077049,Vollmershain,-0.57


### Test Merge

In [21]:
# merge mit Marktstammdaten
marktstd_test = marktstd.merge(bev_entw_raw[['Bev_Entwicklung_%','AGS-8']], on='AGS-8', how='left')
#marktstd_test.head(3)

In [22]:
marktstd_test.isna().sum()

DatumLetzteAktualisierung                      0
NetzbetreiberpruefungStatus                    0
AnlagenbetreiberMastrNummer                    0
Bundesland                                     0
Landkreis                                      0
Gemeinde                                       0
AGS-8                                          0
Postleitzahl                                   0
Ort                                            0
Registrierungsdatum                            0
Inbetriebnahmedatum                            0
EinheitBetriebsstatus                          0
Bruttoleistung                                 0
FernsteuerbarkeitNb                            0
Einspeisungsart                                0
ZugeordneteWirkleistungWechselrichter          0
GemeinsamerWechselrichterMitSpeicher           0
AnzahlModule                                   0
Lage                                           0
Leistungsbegrenzung                            0
EinheitlicheAusricht

In [23]:
# fehlende Werte mit denen des am nächsten verfügbaren AGS 8-stellig ersetzen
marktstd_test = marktstd_test.sort_values(by='AGS-8')
marktstd_filled = marktstd_test.fillna(method='ffill').fillna(method='bfill')
marktstd_filled.isna().sum()
# Quelle: https://stackoverflow.com/questions/38934140/fill-missing-values-by-a-ratio-of-other-values-in-pandas

DatumLetzteAktualisierung                   0
NetzbetreiberpruefungStatus                 0
AnlagenbetreiberMastrNummer                 0
Bundesland                                  0
Landkreis                                   0
Gemeinde                                    0
AGS-8                                       0
Postleitzahl                                0
Ort                                         0
Registrierungsdatum                         0
Inbetriebnahmedatum                         0
EinheitBetriebsstatus                       0
Bruttoleistung                              0
FernsteuerbarkeitNb                         0
Einspeisungsart                             0
ZugeordneteWirkleistungWechselrichter       0
GemeinsamerWechselrichterMitSpeicher        0
AnzahlModule                                0
Lage                                        0
Leistungsbegrenzung                         0
EinheitlicheAusrichtungUndNeigungswinkel    0
Hauptausrichtung                  

In [24]:
bev_entw = marktstd_filled[['AGS-8', 'Bev_Entwicklung_%']].copy()
bev_entw = bev_entw.drop_duplicates()
bev_entw = bev_entw.reset_index()
bev_entw 

,index,AGS-8,Bev_Entwicklung_%
0,2465490,01001000,0.84
1,2414042,01002000,-0.10
2,2032091,01003000,-0.04
3,2438361,01004000,-0.05
4,1095731,01051001,0.27
...,...,...,...
11102,177693,16077048,-0.62
11103,1616665,16077049,-0.57
11104,431049,16077051,-0.57
11105,673774,16077052,-0.72


In [25]:
# nicht benötigte Spalten löschen
bev_entw= bev_entw.drop(['index'], axis=1)
bev_entw

,AGS-8,Bev_Entwicklung_%
0,01001000,0.84
1,01002000,-0.10
2,01003000,-0.04
3,01004000,-0.05
4,01051001,0.27
...,...,...
11102,16077048,-0.62
11103,16077049,-0.57
11104,16077051,-0.57
11105,16077052,-0.72


## verworfen
Bevölkerungsdichte und Grad der Verstädterung nach PLZ (verworfen)
Stand: August 2023
Quelle: https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/Archiv/GVAuszugQ/AuszugGV3QAktuell.html

In [26]:
#bevoelkerung_raw = pd.read_excel("../input/Data/AuszugGV3QAktuell.xlsx", sheet_name="Onlineprodukt_Gemeinden30092023", 
#                                dtype={"Postleitzahl": str, "ARS_1": str, "ARS_2": str, "ARS_3": str, "ARS_4": str})
#bevoelkerung_raw = bevoelkerung_raw.rename(columns={'Unnamed: 12': 'Verstädterung', 'Unnamed: 1': 'ARS_1', 
#                                                    'Unnamed: 2': 'ARS_2', 'Unnamed: 3': 'ARS_3', 'Unnamed: 4': 'ARS_4' })
# bevoelkerung_raw

In [27]:
#count_plz = bevoelkerung_raw['Postleitzahl'].value_counts()
#not_unique = count_plz[count_plz > 1]
# not_unique

**Datenquelle wird verworfen.**   
Um die Tabelle nutzen und mit dem Marktstammdatenregister mergen zu können, wären umfangreiche Transformationsschritte   
notwendig, da die Tabelle viele fehlende Werte enthält und viele Postleitzahlen doppelt oder bis zu 39mal vorkommen.   
Die Bevölkerungsdichte kann zudem aus der vorhergehenden Tabelle entnommen werden.

## verworfen
Eigentümerquote nach Bundesland 
Die Eigentümerquote bezeichnet den Anteil der von Eigentümerinnen und Eigentümern selbst bewohnten Wohnungen   
an allen bewohnten Wohnungen in Wohngebäuden. Sie bezieht sich somit auf die Wohnungsebene.  
Zeitraum: 1998 - 2018
Quelle: https://de.statista.com/statistik/daten/studie/155713/umfrage/anteil-der-buerger-mit-wohneigentum-nach-bundesland/
    
Es soll die Eigentumsquote von 2018 als aktuellster Wert in den Marktstammdaten-Datensatz aufgenommen werden.

In [28]:
# Datensatz einlesen
#dichte_bl_raw = pd.read_excel("../input/Data/statistic_id155713_eigentuemerquote-nach-bundeslaendern-bis-2018.xlsx", sheet_name="Daten", index_col=0)
#dichte_bl_raw

**Tabelle wird zugunsten der detaillierteren Eigenheimquote nach Gemeinde verworfen.**

## Eigenheimquote nach Gemeinde  
Berichtszeitpunkt: 09.05.2011  
Quelle: https://ergebnisse2011.zensus2022.de/datenbank/online/statistic/4000W/table/4000W-1005     
Eigentumsform: Privatpersonen

In [29]:
eigenheimquote_raw = pd.read_csv("../input/Data/4000W-1005_privat_flat.csv" ,encoding="latin1", sep=";",
                                dtype={"1_Auspraegung_Code": str})
# nur Privatpersonen als Ausprägung beibehalten
eigenheimquote_raw = eigenheimquote_raw.loc[eigenheimquote_raw["2_Auspraegung_Label"] == "Privatperson/-en"]
eigenheimquote_raw

,Statistik_Code,Statistik_Label,Zeit_Code,Zeit_Label,Zeit,1_Merkmal_Code,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Code,2_Merkmal_Label,2_Auspraegung_Code,2_Auspraegung_Label,WHG002__ID0004,WHG002__PO0004
1,4000W,Wohnungen,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",GEBEG1,Eigentumsform des Gebäudes,EIG-PRIVAT,Privatperson/-en,23148,"48,2"
3,4000W,Wohnungen,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010020000000,"Kiel, Landeshauptstadt",GEBEG1,Eigentumsform des Gebäudes,EIG-PRIVAT,Privatperson/-en,58908,"44,6"
5,4000W,Wohnungen,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010030000000,"Lübeck, Hansestadt",GEBEG1,Eigentumsform des Gebäudes,EIG-PRIVAT,Privatperson/-en,56872,"49,4"
7,4000W,Wohnungen,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010040000000,"Neumünster, Stadt",GEBEG1,Eigentumsform des Gebäudes,EIG-PRIVAT,Privatperson/-en,22828,"55,5"
9,4000W,Wohnungen,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010510011011,"Brunsbüttel, Stadt",GEBEG1,Eigentumsform des Gebäudes,EIG-PRIVAT,Privatperson/-en,4610,"69,9"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22669,4000W,Wohnungen,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,160775009049,Vollmershain,GEBEG1,Eigentumsform des Gebäudes,EIG-PRIVAT,Privatperson/-en,137,"97,9"
22671,4000W,Wohnungen,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,160775009051,Wildenbörten,GEBEG1,Eigentumsform des Gebäudes,EIG-PRIVAT,Privatperson/-en,123,"84,8"
22673,4000W,Wohnungen,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,160775050012,"Gößnitz, Stadt",GEBEG1,Eigentumsform des Gebäudes,EIG-PRIVAT,Privatperson/-en,1510,"67,0"
22675,4000W,Wohnungen,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,160775050017,Heyersdorf,GEBEG1,Eigentumsform des Gebäudes,EIG-PRIVAT,Privatperson/-en,63,"95,5"


### Data Cleaning

In [30]:
# nicht benötigte Spalten löschen
eigenheimquote_raw = eigenheimquote_raw.drop(columns=["Statistik_Code", "Statistik_Label", "Zeit_Code", "Zeit_Label", "Zeit",
                                          "1_Merkmal_Code", "1_Merkmal_Label", "2_Merkmal_Code", "2_Merkmal_Label",
                                         "2_Auspraegung_Code"])
# eigenheimquote_raw

In [31]:
# Spalten umbenennen
eigenheimquote_raw = eigenheimquote_raw.rename(columns={'1_Auspraegung_Code': 'Regionalschluessel', 
                                            "1_Auspraegung_Label": 'Gemeinde',
                                            "2_Auspraegung_Label": 'Eigentumsform',
                                            "WHG002__ID0004": "Eigentuemer",
                                            "WHG002__PO0004": "Eigentum_%"})

eigenheimquote_raw

,Regionalschluessel,Gemeinde,Eigentumsform,Eigentuemer,Eigentum_%
1,010010000000,"Flensburg, Stadt",Privatperson/-en,23148,"48,2"
3,010020000000,"Kiel, Landeshauptstadt",Privatperson/-en,58908,"44,6"
5,010030000000,"Lübeck, Hansestadt",Privatperson/-en,56872,"49,4"
7,010040000000,"Neumünster, Stadt",Privatperson/-en,22828,"55,5"
9,010510011011,"Brunsbüttel, Stadt",Privatperson/-en,4610,"69,9"
...,...,...,...,...,...
22669,160775009049,Vollmershain,Privatperson/-en,137,"97,9"
22671,160775009051,Wildenbörten,Privatperson/-en,123,"84,8"
22673,160775050012,"Gößnitz, Stadt",Privatperson/-en,1510,"67,0"
22675,160775050017,Heyersdorf,Privatperson/-en,63,"95,5"


In [32]:
# Check: fehlende Werte
eigenheimquote_raw.isna().sum()

Regionalschluessel    0
Gemeinde              0
Eigentumsform         0
Eigentuemer           0
Eigentum_%            0
dtype: int64

In [33]:
count = (eigenheimquote_raw['Eigentuemer'] == '-').sum()
count

1

In [34]:
# Eintrag mit  "-"-werten in Eigentümer und Eigentum mit den Durchschnittswerten ersetzen
eigenheimquote_raw.loc[eigenheimquote_raw["Eigentum_%"] == "-"]
eigenheimquote_raw.loc[eigenheimquote_raw.Eigentuemer == "-", ['Eigentuemer', "Eigentum_%"]] = 1000, 50.0

In [35]:
# eigenheimquote_raw.info()

In [36]:
# Eigentum_% und Eigentuemer formatieren
eigenheimquote_raw["Eigentum_%"] = eigenheimquote_raw["Eigentum_%"].astype(str)
eigenheimquote_raw["Eigentum_%"] = eigenheimquote_raw["Eigentum_%"].str.replace(",", ".").astype(float)
eigenheimquote_raw["Eigentuemer"] = eigenheimquote_raw["Eigentuemer"].astype("int32")
eigenheimquote_raw.head(5)

,Regionalschluessel,Gemeinde,Eigentumsform,Eigentuemer,Eigentum_%
1,010010000000,"Flensburg, Stadt",Privatperson/-en,23148,48.2
3,010020000000,"Kiel, Landeshauptstadt",Privatperson/-en,58908,44.6
5,010030000000,"Lübeck, Hansestadt",Privatperson/-en,56872,49.4
7,010040000000,"Neumünster, Stadt",Privatperson/-en,22828,55.5
9,010510011011,"Brunsbüttel, Stadt",Privatperson/-en,4610,69.9


In [37]:
eigenheimquote_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11339 entries, 1 to 22677
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Regionalschluessel  11339 non-null  object 
 1   Gemeinde            11339 non-null  object 
 2   Eigentumsform       11339 non-null  object 
 3   Eigentuemer         11339 non-null  int32  
 4   Eigentum_%          11339 non-null  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 487.2+ KB


### Gemeindeschlüssel (8-stellig) erzeugen  
Um die Tabelle mit dem Marktstammdatenregister mergen zu können, wird aus dem Regionalschlüssel ein Gemeindeschlüssel extrahiert:    
**Amtlicher Gemeindeschlüssel (AGS)**   
8-stelliger Schlüssel zur eindeutigen Identifizierung einer Gemeinde mit den Bestandteilen:  
Bundesland (2 Stellen), Regierungsbezirk (1 Stelle), Kreis (2 Stellen) und Gemeinde (3 Stellen).  
**Regionalschlüssel (RS)**   
12-stelliger Schlüssel zur eindeutigen Identifizierung einer Gemeinde mit den Bestandteilen: Bundesland (2 Stellen),      Regierungsbezirk (1 Stelle), Kreis (2 Stellen), Gemeindeverband (4 Stellen) und Gemeinde (3 Stellen).   

In [38]:
# von den 12-stelligen Regionalschlüsseln werden die 4 Stellen für Regionalverband gelöscht,    
# um einen 8-stelligen Gemeindeschlüssel zu erzeugen.
eigenheimquote_raw["Regionalschluessel"] = eigenheimquote_raw["Regionalschluessel"].astype(str)
eigenheimquote_raw['AGS-8'] = eigenheimquote_raw['Regionalschluessel'].str[:5] + eigenheimquote_raw['Regionalschluessel'].str[9:]
eigenheimquote_raw

,Regionalschluessel,Gemeinde,Eigentumsform,Eigentuemer,Eigentum_%,AGS-8
1,010010000000,"Flensburg, Stadt",Privatperson/-en,23148,48.2,01001000
3,010020000000,"Kiel, Landeshauptstadt",Privatperson/-en,58908,44.6,01002000
5,010030000000,"Lübeck, Hansestadt",Privatperson/-en,56872,49.4,01003000
7,010040000000,"Neumünster, Stadt",Privatperson/-en,22828,55.5,01004000
9,010510011011,"Brunsbüttel, Stadt",Privatperson/-en,4610,69.9,01051011
...,...,...,...,...,...,...
22669,160775009049,Vollmershain,Privatperson/-en,137,97.9,16077049
22671,160775009051,Wildenbörten,Privatperson/-en,123,84.8,16077051
22673,160775050012,"Gößnitz, Stadt",Privatperson/-en,1510,67.0,16077012
22675,160775050017,Heyersdorf,Privatperson/-en,63,95.5,16077017


In [39]:
pd.Series(eigenheimquote_raw["AGS-8"]).is_unique

True

In [40]:
eigenheimquote_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11339 entries, 1 to 22677
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Regionalschluessel  11339 non-null  object 
 1   Gemeinde            11339 non-null  object 
 2   Eigentumsform       11339 non-null  object 
 3   Eigentuemer         11339 non-null  int32  
 4   Eigentum_%          11339 non-null  float64
 5   AGS-8               11339 non-null  object 
dtypes: float64(1), int32(1), object(4)
memory usage: 575.8+ KB


In [41]:
# nicht benötigte Spalten löschen
drop_cols = ["Regionalschluessel", "Gemeinde", "Eigentumsform"]
eigenheimquote_raw = eigenheimquote_raw.drop(drop_cols, axis=1)
eigenheimquote_raw

,Eigentuemer,Eigentum_%,AGS-8
1,23148,48.2,01001000
3,58908,44.6,01002000
5,56872,49.4,01003000
7,22828,55.5,01004000
9,4610,69.9,01051011
...,...,...,...
22669,137,97.9,16077049
22671,123,84.8,16077051
22673,1510,67.0,16077012
22675,63,95.5,16077017


### Test Merge

In [42]:
# Test merge mit Marktstammdaten
marktstd_test = marktstd.merge(eigenheimquote_raw, on='AGS-8', how='left')
marktstd_test

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,...,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,AGS-5,Inbetriebnahmejahr,Eigentuemer,Eigentum_%
0,2020-02-20 16:28:35.250812200,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01,...,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,20 - 40 Grad,Haushalt,05515,2007,80284.0,51.8
1,2021-07-15 07:54:37.863709500,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,...,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,08136,2013,16141.0,57.1
2,2019-05-28 08:26:38.031669200,1,ABR983885950021,Brandenburg,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,...,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,12063,2016,4892.0,56.7
3,2020-12-07 08:19:07.986079300,1,ABR980052996196,Bayern,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,...,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, sonstige",1.0,Süd-West,20 - 40 Grad,Haushalt,09375,2016,2359.0,85.4
4,2021-01-25 11:18:30.043360300,1,ABR970373690446,Saarland,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,...,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,0.0,West,20 - 40 Grad,Haushalt,10044,2011,12988.0,72.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173005,2020-07-14 07:21:05.450956000,1,ABR941126007873,Nordrhein-Westfalen,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,...,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Ost,< 20 Grad,Haushalt,05970,2009,5216.0,80.1
3173006,2020-04-29 09:37:03.402485600,0,ABR903375397990,Nordrhein-Westfalen,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,...,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,40 - 60 Grad,Landwirtschaft,05958,2010,4139.0,78.5
3173007,2020-11-26 08:57:11.006049400,1,ABR985070432688,Bayern,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,...,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",0.0,West,20 - 40 Grad,Haushalt,09374,2020,409.0,90.3
3173008,2020-05-02 10:16:32.781845900,1,ABR925833676461,Hessen,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,...,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,06433,2016,6859.0,73.1


In [43]:
# jeweils erste Zeile AGS 8-stellig behalten / Duplikate verwerfen
marktstd_test = marktstd_test.drop_duplicates(subset='AGS-8', keep='first')

In [44]:
marktstd_test.isna().sum()

DatumLetzteAktualisierung                     0
NetzbetreiberpruefungStatus                   0
AnlagenbetreiberMastrNummer                   0
Bundesland                                    0
Landkreis                                     0
Gemeinde                                      0
AGS-8                                         0
Postleitzahl                                  0
Ort                                           0
Registrierungsdatum                           0
Inbetriebnahmedatum                           0
EinheitBetriebsstatus                         0
Bruttoleistung                                0
FernsteuerbarkeitNb                           0
Einspeisungsart                               0
ZugeordneteWirkleistungWechselrichter         0
GemeinsamerWechselrichterMitSpeicher          0
AnzahlModule                                  0
Lage                                          0
Leistungsbegrenzung                           0
EinheitlicheAusrichtungUndNeigungswinkel

In [45]:
# in welchen AGS 8-stellig fehlen Werte?
loc_missings = marktstd_test[marktstd_test['Eigentum_%'].isnull()]['AGS-8'].unique()
loc_missings

array(['13075015', '01059187', '03352062', '13075039', '03159017',
       '03153017', '13073007', '13071107', '13075053', '13071110',
       '13071069', '14521355', '13074074', '13072029', '13074021',
       '16063000', '03153018', '13074019', '13072043', '03456027',
       '13076094', '16076092', '13073038', '03153016', '03352061',
       '13074016', '13074057', '14628205', '13071127', '03154028',
       '13076152', '01059185', '13072001', '03159018', '13071156',
       '03159026', '03352059', '03251049', '13073074', '13076105',
       '16064074', '03157009', '13073093', '13075009', '16068064',
       '13076078', '03155013', '03351026', '13074084', '13071033',
       '13073060', '03159010', '13075044', '13075079', '13075025',
       '13076156', '13073103', '13071162', '13073088', '03159034',
       '13071060', '13075018', '16063099', '16069062', '13074032',
       '13073057', '13072105', '13073035', '13074026', '03352060',
       '13076033', '16063101', '03360030', '13071122', '130730

Die deutlich größte Anzahl von fehlenden Werten gibt es bei AGS 8-stellig, die mit 13 beginnen,   
also in **Mecklenburg-Vorpommern** liegen.   
Durchschnittswert für Mecklenburg-Vorpommern: **49,5 %**   
(Quelle: https://ergebnisse2011.zensus2022.de/datenbank//online?operation=table&code=4000W-1005&bypass=true&levelindex=0&levelid=1653243306394#abreadcrumb)

In [46]:
# Alle fehlenden Werte für die Eigentümerquote werden mit 49,5% ersetzt.
# Die Anzahl der Eigentümer wird gelöscht, da fehlende Werte schwierig zu ersetzen sind und die   
# Wohneigentums-Quote deutlich aussagekräftiger ist.
marktstd_test['Eigentum_%'] = marktstd_test['Eigentum_%'].fillna(49.5)
marktstd_test = marktstd_test.drop('Eigentuemer', axis=1)
marktstd_test.isna().sum()

C:\Users\Conny\AppData\Local\Temp\ipykernel_8880\1123973450.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marktstd_test['Eigentum_%'] = marktstd_test['Eigentum_%'].fillna(49.5)


DatumLetzteAktualisierung                   0
NetzbetreiberpruefungStatus                 0
AnlagenbetreiberMastrNummer                 0
Bundesland                                  0
Landkreis                                   0
Gemeinde                                    0
AGS-8                                       0
Postleitzahl                                0
Ort                                         0
Registrierungsdatum                         0
Inbetriebnahmedatum                         0
EinheitBetriebsstatus                       0
Bruttoleistung                              0
FernsteuerbarkeitNb                         0
Einspeisungsart                             0
ZugeordneteWirkleistungWechselrichter       0
GemeinsamerWechselrichterMitSpeicher        0
AnzahlModule                                0
Lage                                        0
Leistungsbegrenzung                         0
EinheitlicheAusrichtungUndNeigungswinkel    0
Hauptausrichtung                  

In [47]:
eigenheimquote = marktstd_test[['Eigentum_%', 'AGS-8']].reset_index()
eigenheimquote

,index,Eigentum_%,AGS-8
0,0,51.8,05515000
1,1,57.1,08136065
2,2,56.7,12063208
3,3,85.4,09375180
4,4,72.2,10044115
...,...,...,...
11102,2898153,86.7,16063019
11103,3078866,80.6,07232038
11104,3113783,81.0,14729230
11105,3142044,68.3,16075069


In [48]:
# nicht benötigte Spalten löschen
# bereinigte Wohneigentumsquote als Tabelle speichern
eigenheimquote= eigenheimquote.drop(['index'], axis=1)
eigenheimquote

,Eigentum_%,AGS-8
0,51.8,05515000
1,57.1,08136065
2,56.7,12063208
3,85.4,09375180
4,72.2,10044115
...,...,...
11102,86.7,16063019
11103,80.6,07232038
11104,81.0,14729230
11105,68.3,16075069


## Gini-Koeffizient nach Bundesland

Der Gini-Koeffizient oder Gini-Index ist ein Maß der relativen Konzentration beziehungsweise Ungleichheit und kann einen Wert zwischen Null und Eins (Gini-Koeffizient) bzw.    
skaliert von 0 bis 100 (Gini-Index) annehmen. Im Falle der Gleich­verteilung ergibt sich für den Gini-Koeffizienten ein Wert von Null und im Falle der Konzentration des    
gesamten Einkommens auf nur eine Person ein Wert von 1.

Quelle: https://www.statistikportal.de/de/sbe/ergebnisse/einkommen-armutsgefaehrdung-und-soziale-lebensbedingungen/armutsgefaehrdung-und-3

In [49]:
# Datensatz 1 (bis 2019) einlesen
gini_raw_1 = pd.read_excel("../input/Data/A12 Gini Koeffizient bis 2019.xlsx", index_col=0)
# gini_raw_1

In [50]:
# Datensatz 2 (ab 2019) einlesen
gini_raw_2 = pd.read_excel("../input/Data/A12 Gini Koeffizient ab 2020.xlsx", index_col=0)
# gini_raw_2

In [51]:
gini_raw = pd.merge(gini_raw_1, gini_raw_2, left_index=True, right_index=True)
# gini_raw

### Data Cleaning

Für Gesamt-Deutschland ist über die Jahre wenig Veränderung des Gini-Koeffizienten zu beobachten,  es gibt jedoch eine insgesamt leicht steigende Tendenz.   
Zwischen den Bundesländern gibt es durchaus Unterschiede.    
Nierigster Wert: Mecklenburg-Vorpommern mit durchschnittlich 0,26    
Höchster Wert: Hamburg mit durchschnittlich 0,32    
Für weitere Analysen wird der aktuelle Wert von 2023 betrachtet.

In [52]:
gini_raw['Durchschnitt_Gini'] = gini_raw.mean(numeric_only=True, axis=1).round(2)
gini_raw

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,20202,2021,2023,Durchschnitt_Gini
Baden-Württemberg,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29
Bayern,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.30,0.30,0.29
Berlin,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.29,0.30,0.29,0.30,0.30,0.30,0.30,0.30,0.32,0.32,0.31,0.30
Brandenburg,0.27,0.26,0.25,0.26,0.26,0.27,0.27,0.27,0.27,0.27,0.27,0.26,0.27,0.26,0.26,0.28,0.28,0.26,0.27
Bremen,0.30,0.29,0.29,0.29,0.30,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.32,0.33,0.31,0.31
Hamburg,0.32,0.31,0.32,0.32,0.33,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.31,0.31,0.33,0.33,0.32
Hessen,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.31,0.31,0.30,0.31,0.31,0.31,0.31,0.31,0.32,0.32,0.31,0.31
Mecklenburg-Vorpommern,0.26,0.25,0.25,0.25,0.26,0.26,0.26,0.26,0.26,0.25,0.26,0.27,0.27,0.26,0.26,0.26,0.25,0.25,0.26
Niedersachsen,0.29,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.29,0.30,0.30,0.28
Nordrhein-Westfalen,0.29,0.29,0.29,0.29,0.29,0.29,0.30,0.29,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.32,0.31,0.30


In [53]:
# Welche Spalten weisen Nan-Values auf? 
# gini_raw.isna().sum()

In [54]:
gini_raw = gini_raw.reset_index()
# Spalten umbenennen
gini_raw = gini_raw.rename(columns={'index': 'Bundesland', 2023 : 'Gini_Index'})
gini_raw['Bundesland'] = gini_raw['Bundesland'].astype(str)
# gini_raw

In [55]:
# nicht benötigte Spalten löschen
gini_raw = gini_raw.drop(['Durchschnitt_Gini'], axis=1)
gini_raw = gini_raw.drop([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 20202, 2021], axis=1)
gini_raw

,Bundesland,Gini_Index
0,Baden-Württemberg,0.29
1,Bayern,0.30
2,Berlin,0.31
3,Brandenburg,0.26
4,Bremen,0.31
5,Hamburg,0.33
6,Hessen,0.31
7,Mecklenburg-Vorpommern,0.25
8,Niedersachsen,0.30
9,Nordrhein-Westfalen,0.31


In [56]:
gini_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Bundesland  19 non-null     object 
 1   Gini_Index  19 non-null     float64
dtypes: float64(1), object(1)
memory usage: 436.0+ bytes


### Test Merge

Der **Gini-Index** hat sich für die einzelnen Bundesländer zwischen 2005 und 2023 nicht sehr verändert.    
Der Unterschied zwischen den Bundesländern ist jedoch aussagekräftig.   
Insofern werden die aktuellen Werte von 2023 als Gini Index übernommen und testweise mit den Marktstammdaten gemergt.

In [57]:
# TEST merge mit Marktstammdaten
#marktstd = marktstd.merge(gini_raw, on="Bundesland", how="left")
#marktstd.head()

In [58]:
# Welche Spalten weisen Nan-Values auf? 
#marktstd.isna().sum()

In [59]:
#marktstd.describe()

## Verfügbares Einkommen der Privathaushalte
Das Verfügbare Einkommen der privaten Haushalte einschließlich privater Organisationen ohne Erwerbszweck (Ausgabenkonzept) ergibt sich dadurch,   
dass dem Primäreinkommen einerseits die monetären Sozialleistungen und sonstigen laufenden Transfers hinzugefügt werden, die die privaten Haushalte   
überwiegend seitens des Staates empfangen; abgezogen werden dagegen andererseits Einkommen- und Vermögensteuern, Sozialbeiträge und sonstige laufende   
Transfers, die von den privaten Haushalten zu leisten sind. Das Verfügbare Einkommen der privaten Haushalte einschließlich privater Organisationen   ohne Erwerbszweck entspricht damit den Einkommen, die den privaten Haushalten letztendlich zufließen und die sie für Konsum- und Sparzwecke verwenden    können.

Verfügbares Einkommen der privaten Haushalte einschl. der privaten Organisationen ohne Erwerbszweck je Einwohner   
in EUR    
Zeitraum: 1995 bis 2020   
Quelle: https://www.statistikportal.de/de/veroeffentlichungen/einkommen-der-privaten-haushalte   
(Excel Sheet 2.4)

In [60]:
einkommen_raw = pd.read_excel("../input/Data/vgrdl_r2b3_bs2021_1.xlsx", sheet_name="2.4", dtype={'Regional-schlüssel' : str}, index_col=0)
einkommen_raw

,EU-Code,Regional-schlüssel,Land,NUTS 1,NUTS 2,NUTS 3,Gebietseinheit,1995,1996,1997,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Lfd. Nr.,,,,,,,,,,,,,,,,,,,,,
1,DE1,08,BW,1.0,NaN,NaN,Baden-Württemberg,15430,15736,16060,...,21609,22128,22365,22920,23206,23755,24467,25404,25739,25513
2,DE11,081,BW,NaN,2.0,NaN,"Stuttgart, Regierungsbezirk",16062,16370,16722,...,22394,22904,23116,23620,23836,24456,25304,25918,26267,26047
3,DE111,08111,BW,NaN,NaN,3.0,"Stuttgart, Landeshauptstadt, Stadtkreis",17883,18210,18624,...,23121,23487,23555,24222,24358,24788,25665,26561,26749,26503
4,DE112,08115,BW,NaN,NaN,3.0,"Böblingen, Landkreis",16465,16898,17552,...,22364,23059,23261,23741,24248,24481,25065,26199,26737,26608
5,DE113,08116,BW,NaN,NaN,3.0,"Esslingen, Landkreis",17167,17472,17803,...,22526,23139,23462,23950,24243,24454,25308,26321,26729,26577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,DEG0L,16076,TH,NaN,NaN,3.0,"Greiz, Landkreis",10303,10806,11323,...,16816,17171,17607,17853,18352,19010,19908,20534,21175,21561
442,DEG0M,16077,TH,NaN,NaN,3.0,"Altenburger Land, Landkreis",11226,11569,11627,...,16045,16445,16870,17130,17684,18480,19320,20013,20611,21154
443,DEG0N,16056,TH,NaN,NaN,3.0,"Eisenach, Kreisfreie Stadt1)",11170,11854,11960,...,16666,17045,17472,17961,18282,18664,19225,19811,20314,20742


### Data Cleaning

In [61]:
# nicht benötigte Spalten löschen
einkommen_raw = einkommen_raw.drop(['EU-Code','Land','NUTS 1','NUTS 2', 'NUTS 3'], axis=1)
# einkommen_raw

In [62]:
# Regionalschlüssel als AGS 5-stellig formatieren und Spalte umbenennen
einkommen_raw['Regional-schlüssel'] = einkommen_raw['Regional-schlüssel'].str.ljust(5, '0')
einkommen_raw["Regional-schlüssel"] = einkommen_raw["Regional-schlüssel"].astype(str)
einkommen_raw = einkommen_raw.rename(columns={'Regional-schlüssel': 'AGS-5'})
einkommen_raw.head()

,AGS-5,Gebietseinheit,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Lfd. Nr.,,,,,,,,,,,,,,,,,,,,,
1,08000,Baden-Württemberg,15430,15736,16060,16328,16794,17006,17911,17844,...,21609,22128,22365,22920,23206,23755,24467,25404,25739,25513
2,08100,"Stuttgart, Regierungsbezirk",16062,16370,16722,17028,17474,17692,18595,18511,...,22394,22904,23116,23620,23836,24456,25304,25918,26267,26047
3,08111,"Stuttgart, Landeshauptstadt, Stadtkreis",17883,18210,18624,18928,19376,19326,20196,20087,...,23121,23487,23555,24222,24358,24788,25665,26561,26749,26503
4,08115,"Böblingen, Landkreis",16465,16898,17552,18002,18277,18335,19130,19058,...,22364,23059,23261,23741,24248,24481,25065,26199,26737,26608
5,08116,"Esslingen, Landkreis",17167,17472,17803,18079,18559,18848,19792,19610,...,22526,23139,23462,23950,24243,24454,25308,26321,26729,26577


In [63]:
# Merge mit Marktstammdaten / AGS 5-stellig
subset = marktstd[['AGS-5']]
einkommen = einkommen_raw.merge(subset, on="AGS-5", how="left")
einkommen.head()

,AGS-5,Gebietseinheit,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,08000,Baden-Württemberg,15430,15736,16060,16328,16794,17006,17911,17844,...,21609,22128,22365,22920,23206,23755,24467,25404,25739,25513
1,08100,"Stuttgart, Regierungsbezirk",16062,16370,16722,17028,17474,17692,18595,18511,...,22394,22904,23116,23620,23836,24456,25304,25918,26267,26047
2,08111,"Stuttgart, Landeshauptstadt, Stadtkreis",17883,18210,18624,18928,19376,19326,20196,20087,...,23121,23487,23555,24222,24358,24788,25665,26561,26749,26503
3,08111,"Stuttgart, Landeshauptstadt, Stadtkreis",17883,18210,18624,18928,19376,19326,20196,20087,...,23121,23487,23555,24222,24358,24788,25665,26561,26749,26503
4,08111,"Stuttgart, Landeshauptstadt, Stadtkreis",17883,18210,18624,18928,19376,19326,20196,20087,...,23121,23487,23555,24222,24358,24788,25665,26561,26749,26503


In [64]:
# nach Duplikaten suchen
duplicate_rows = einkommen[einkommen.duplicated()].count()
#duplicate_rows

In [65]:
# Duplikate löschen
einkommen = einkommen.drop_duplicates().reset_index()
#einkommen

In [66]:
einkommen = einkommen.drop(['index', 'Gebietseinheit'], axis=1)
einkommen

,AGS-5,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,08000,15430,15736,16060,16328,16794,17006,17911,17844,18315,...,21609,22128,22365,22920,23206,23755,24467,25404,25739,25513
1,08100,16062,16370,16722,17028,17474,17692,18595,18511,19003,...,22394,22904,23116,23620,23836,24456,25304,25918,26267,26047
2,08111,17883,18210,18624,18928,19376,19326,20196,20087,20729,...,23121,23487,23555,24222,24358,24788,25665,26561,26749,26503
3,08115,16465,16898,17552,18002,18277,18335,19130,19058,19449,...,22364,23059,23261,23741,24248,24481,25065,26199,26737,26608
4,08116,17167,17472,17803,18079,18559,18848,19792,19610,20003,...,22526,23139,23462,23950,24243,24454,25308,26321,26729,26577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,16076,10303,10806,11323,11731,12341,12861,13511,13711,14039,...,16816,17171,17607,17853,18352,19010,19908,20534,21175,21561
441,16077,11226,11569,11627,11947,12449,12694,13253,13489,13746,...,16045,16445,16870,17130,17684,18480,19320,20013,20611,21154
442,16056,11170,11854,11960,12521,13146,13386,13863,14319,14695,...,16666,17045,17472,17961,18282,18664,19225,19811,20314,20742
443,16063,10922,11454,11652,12156,12823,13146,13757,13920,14222,...,17126,17647,18231,18286,18938,19748,20266,21228,21603,22098


In [67]:
# Spalten umbenennen
einkommen.columns = ['AGS-5'] + ['haushaltseink_' + str(col) 
                                 for col in einkommen.columns[1:]]
einkommen

,AGS-5,haushaltseink_1995,haushaltseink_1996,haushaltseink_1997,haushaltseink_1998,haushaltseink_1999,haushaltseink_2000,haushaltseink_2001,haushaltseink_2002,haushaltseink_2003,...,haushaltseink_2011,haushaltseink_2012,haushaltseink_2013,haushaltseink_2014,haushaltseink_2015,haushaltseink_2016,haushaltseink_2017,haushaltseink_2018,haushaltseink_2019,haushaltseink_2020
0,08000,15430,15736,16060,16328,16794,17006,17911,17844,18315,...,21609,22128,22365,22920,23206,23755,24467,25404,25739,25513
1,08100,16062,16370,16722,17028,17474,17692,18595,18511,19003,...,22394,22904,23116,23620,23836,24456,25304,25918,26267,26047
2,08111,17883,18210,18624,18928,19376,19326,20196,20087,20729,...,23121,23487,23555,24222,24358,24788,25665,26561,26749,26503
3,08115,16465,16898,17552,18002,18277,18335,19130,19058,19449,...,22364,23059,23261,23741,24248,24481,25065,26199,26737,26608
4,08116,17167,17472,17803,18079,18559,18848,19792,19610,20003,...,22526,23139,23462,23950,24243,24454,25308,26321,26729,26577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,16076,10303,10806,11323,11731,12341,12861,13511,13711,14039,...,16816,17171,17607,17853,18352,19010,19908,20534,21175,21561
441,16077,11226,11569,11627,11947,12449,12694,13253,13489,13746,...,16045,16445,16870,17130,17684,18480,19320,20013,20611,21154
442,16056,11170,11854,11960,12521,13146,13386,13863,14319,14695,...,16666,17045,17472,17961,18282,18664,19225,19811,20314,20742
443,16063,10922,11454,11652,12156,12823,13146,13757,13920,14222,...,17126,17647,18231,18286,18938,19748,20266,21228,21603,22098


### Test Merge

In [68]:
# TEST merge mit Marktstammdaten
#pd.set_option('display.max_columns', 100)
#marktstd = marktstd.merge(einkommen_raw, on="AGS-5", how="left")
#marktstd.head()

In [69]:
#pd.set_option('display.max_rows', 100)
#marktstd.isna().sum()

## Bruttoinlandsprodukt BIP
   
Die Daten der Volkswirtschaftlichen Gesamtrechnung (VGR) unterliegen regelmäßigen Revisionen, Berechnungsstand ist hier November 2021. Für die Berechnung des Indikators wird der Wert des Bruttoinlandsprodukts in jeweiligen Preisen im Jahr durch die Zahl der Erwerbstätigen (am Arbeitsort) im Jahresdurchschnitt dividiert.   
   
Tabelle: Deutschlandatlas_KRS1221.csv   
Quelle: https://www.deutschlandatlas.bund.de/DE/Service/Downloads/downloads_node.html    

**BIP**: Bruttoinlandsprodukt (BIP) je erwerbstätige Person im Jahr 2021 in 1.000 €   

In [70]:
# Datensatz einlesen
columns = ['Gebietskennziffer', 'name', 'erw_bip']
bip_raw = pd.read_csv('../input/Deutschlandatlas/Deutschlandatlas_KRS1220.csv', usecols=columns, encoding='latin1', 
                             dtype={'Gebietskennziffer' : str}, sep=";", decimal=",", low_memory=False)
bip_raw = bip_raw.rename(columns={'erw_bip': 'BIP', 'name' : 'Ort'})
bip_raw

,Gebietskennziffer,Ort,BIP
0,1001000,"Flensburg, Stadt",63.55
1,1002000,"Kiel, Stadt",68.26
2,1003000,"Lübeck, Stadt",72.59
3,1004000,"Neumünster, Stadt",65.92
4,1051000,Dithmarschen,72.64
...,...,...,...
396,16073000,Saalfeld-Rudolstadt,60.07
397,16074000,Saale-Holzland-Kreis,59.98
398,16075000,Saale-Orla-Kreis,64.25
399,16076000,Greiz,58.67


### Data Cleaning

In [71]:
# Gebietskennziffer umwandeln in Gemeindeschlüssel AGS 5-stellig
bip_raw['Gebietskennziffer'] = bip_raw['Gebietskennziffer'].apply(lambda x: '0' + x if len(x) == 7 else x)
bip_raw['AGS-5'] = bip_raw['Gebietskennziffer'].str[:-3]
bip_raw
# Quelle: ChatGPT (siehe ChatGPT Protokoll, Seite 3)

,Gebietskennziffer,Ort,BIP,AGS-5
0,01001000,"Flensburg, Stadt",63.55,01001
1,01002000,"Kiel, Stadt",68.26,01002
2,01003000,"Lübeck, Stadt",72.59,01003
3,01004000,"Neumünster, Stadt",65.92,01004
4,01051000,Dithmarschen,72.64,01051
...,...,...,...,...
396,16073000,Saalfeld-Rudolstadt,60.07,16073
397,16074000,Saale-Holzland-Kreis,59.98,16074
398,16075000,Saale-Orla-Kreis,64.25,16075
399,16076000,Greiz,58.67,16076


In [72]:
# nicht benötigte Spalten löschen
bip_raw = bip_raw.drop(['Gebietskennziffer', 'Ort'], axis=1)
bip_raw

,BIP,AGS-5
0,63.55,01001
1,68.26,01002
2,72.59,01003
3,65.92,01004
4,72.64,01051
...,...,...
396,60.07,16073
397,59.98,16074
398,64.25,16075
399,58.67,16076


### Test Merge

In [73]:
# merge mit Marktstammdaten
#marktstd = marktstd.merge(bip_raw[['BIP', 'AGS-5']], on='AGS-5', how='left')
#marktstd

In [74]:
#marktstd.isna().sum()

## Arbeitslosenquote nach Bundesland
Arbeitslosenquote, bezogen auf alle zivilen Erwerbspersonen:
Die zivilen Erwerbspersonen setzen sich aus den zivilen Erwerbstätigen und den re­gis­trier­ten Ar­beits­losen zu­sam­men.   
Die zivilen Erwerbstätigen er­ge­ben sich aus der Sum­me der ab­hän­gigen zivilen Erwerbstätigen (analog der o.a. Be­schrei­bung) 
sowie der Selbst­stän­di­gen und mithelfenden Fa­mi­lien­an­ge­hö­ri­gen aus dem Mikrozensus.   
   
Arbeitlosenquote pro Jahr und Bundesland, beginnend im Jahr 1991    
Quelle: https://www.govdata.de/web/guest/suchen/-/details/arbeitslose-arbeitslosenquoten-gemeldete-arbeitsstellen-bundeslander-jahre  

In [75]:
# Datensatz einlesen
arbeitslos_raw = pd.read_excel('../input/Data/Arbeitslosenquote.xlsx', index_col=0)
arbeitslos_raw

,Jahr,Anzahl Arbeitslose,"Arbeitslosenquote aller zivilen Erwerbspersonen, %","Arbeitslosenquote d. abhängigen ziv. Erwerbspers., %",Anzahl Gemeldete Arbeitsstellen
Bundesland,,,,,
Baden-Württemberg,1991,159318,-,3.7,77904
Baden-Württemberg,1992,191970,-,4.4,64122
Baden-Württemberg,1993,281496,-,6.3,37952
Baden-Württemberg,1994,333416,6.7,7.5,37827
Baden-Württemberg,1995,328298,6.6,7.4,44777
...,...,...,...,...,...
Thüringen,2018,62145,5.5,6.0,25916
Thüringen,2019,59065,5.3,5.7,23411
Thüringen,2020,66678,6,6.5,18211


### Data Cleaning

In [76]:
# nicht benötigte Spalten löschen
drop_cols = ['Anzahl Arbeitslose', 'Arbeitslosenquote d. abhängigen ziv. Erwerbspers., %', 'Anzahl Gemeldete Arbeitsstellen']
arbeitslos_raw = arbeitslos_raw.drop(drop_cols, axis=1)
# arbeitslos_raw

In [77]:
# fehlende Werte mit None ersetzen
arbeitslos_raw.replace({'-': None})

,Jahr,"Arbeitslosenquote aller zivilen Erwerbspersonen, %"
Bundesland,,
Baden-Württemberg,1991,None
Baden-Württemberg,1992,None
Baden-Württemberg,1993,None
Baden-Württemberg,1994,6.7
Baden-Württemberg,1995,6.6
...,...,...
Thüringen,2018,5.5
Thüringen,2019,5.3
Thüringen,2020,6


In [78]:
arbeitslos_raw = arbeitslos_raw[arbeitslos_raw['Jahr'] == 2022]
arbeitslos_raw = arbeitslos_raw.reset_index()
arbeitslos_raw = arbeitslos_raw.drop(columns=["Jahr"])
arbeitslos_raw = arbeitslos_raw.rename(columns={'Arbeitslosenquote aller zivilen Erwerbspersonen, %': 'Arbeitslosenquote_%'})
#arbeitslos_raw

Ähnlich wie beim Gini-Index soll vor allem eine Vergleichbarkeit zwischen den Bundesländern erreicht werden.    
Daher wird die aktuellste **Arbeitslosenquote von 2022** für jedes Bundesland übernommen.  

In [79]:
arbeitslos_raw['Arbeitslosenquote_%'] = arbeitslos_raw['Arbeitslosenquote_%'].astype(float)
arbeitslos_raw

,Bundesland,Arbeitslosenquote_%
0,Baden-Württemberg,3.5
1,Bayern,3.1
2,Berlin,8.8
3,Brandenburg,5.6
4,Bremen,10.2
5,Hamburg,6.8
6,Hessen,4.8
7,Mecklenburg-Vorpommern,7.3
8,Niedersachsen,5.3
9,Nordrhein-Westfalen,6.8


### Test Merge

In [80]:
# merge mit Marktstammdaten
#marktstd = marktstd.merge(arbeitslos_raw, on="Bundesland", how="left")
#marktstd.head()

In [81]:
# Welche Spalten weisen Nan-Values auf? 
#marktstd.isna().sum()

## Altersstruktur in Prozent
Berichtszeitpunkt: 09.05.2011     
Tabelle:  1000A-1005, Personen: Alter (Infrastrukturelle Jahresgruppen)       
Quelle: https://ergebnisse2011.zensus2022.de/datenbank/online/statistic/1000A/table/1000A-1005

In [82]:
alter_raw = pd.read_csv("../input/Data/1000A-1005_flat.csv" ,encoding="latin1", sep=";", low_memory=False)
alter_raw

,Statistik_Code,Statistik_Label,Zeit_Code,Zeit_Label,Zeit,1_Merkmal_Code,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Code,2_Merkmal_Label,2_Auspraegung_Code,2_Auspraegung_Label,PRS001__ID0004,PRS001__PO0004,PRS018__ID0006,PRS018__PO0006
0,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTGR3,Alter (Infrastrukturelle Altersgruppen),NaN,Insgesamt,NaN,NaN,82258,"100,0"
1,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTGR3,Alter (Infrastrukturelle Altersgruppen),ALT000B002,Unter 3 Jahre,NaN,NaN,1933,"2,3"
2,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTGR3,Alter (Infrastrukturelle Altersgruppen),ALT003B005,3 bis 5 Jahre,NaN,NaN,1804,"2,2"
3,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTGR3,Alter (Infrastrukturelle Altersgruppen),ALT006B009,6 bis 9 Jahre,NaN,NaN,2662,"3,2"
4,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTGR3,Alter (Infrastrukturelle Altersgruppen),ALT010B015,10 bis 15 Jahre,NaN,NaN,4471,"5,4"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136075,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEODL1,Deutschland,DG,Deutschland,ALTGR3,Alter (Infrastrukturelle Altersgruppen),ALT025B039,25 bis 39 Jahre,14367337.0,"17,9",NaN,NaN
136076,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEODL1,Deutschland,DG,Deutschland,ALTGR3,Alter (Infrastrukturelle Altersgruppen),ALT040B059,40 bis 59 Jahre,24976611.0,"31,1",NaN,NaN
136077,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEODL1,Deutschland,DG,Deutschland,ALTGR3,Alter (Infrastrukturelle Altersgruppen),ALT060B066,60 bis 66 Jahre,6108259.0,"7,6",NaN,NaN
136078,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEODL1,Deutschland,DG,Deutschland,ALTGR3,Alter (Infrastrukturelle Altersgruppen),ALT067B074,67 bis 74 Jahre,7629146.0,"9,5",NaN,NaN


### Data Cleaning

In [83]:
# Spalten löschen
drop_cols = ['Statistik_Code', 'Statistik_Label', 'Zeit_Code', 'Zeit_Label', 'Zeit', '1_Merkmal_Code',
            '2_Merkmal_Code', '2_Auspraegung_Code', 'PRS001__PO0004', 'PRS001__ID0004' ]
alter_raw = alter_raw.drop(drop_cols, axis=1)
alter_raw.head()

,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Label,2_Auspraegung_Label,PRS018__ID0006,PRS018__PO0006
0,Gemeinden,010010000000,"Flensburg, Stadt",Alter (Infrastrukturelle Altersgruppen),Insgesamt,82258,"100,0"
1,Gemeinden,010010000000,"Flensburg, Stadt",Alter (Infrastrukturelle Altersgruppen),Unter 3 Jahre,1933,"2,3"
2,Gemeinden,010010000000,"Flensburg, Stadt",Alter (Infrastrukturelle Altersgruppen),3 bis 5 Jahre,1804,"2,2"
3,Gemeinden,010010000000,"Flensburg, Stadt",Alter (Infrastrukturelle Altersgruppen),6 bis 9 Jahre,2662,"3,2"
4,Gemeinden,010010000000,"Flensburg, Stadt",Alter (Infrastrukturelle Altersgruppen),10 bis 15 Jahre,4471,"5,4"


In [84]:
# nur Gemeinden behalten / Daten für gesamt Deutschland löschen
alter_raw = alter_raw[alter_raw['1_Merkmal_Label'].isin(['Gemeinden'])]
# alter_raw

In [85]:
# Spalten löschen / umbenennen
alter_raw = alter_raw.drop(['1_Merkmal_Label', '1_Auspraegung_Label', '2_Merkmal_Label', 'PRS018__ID0006'], axis=1)
alter_raw = alter_raw.rename(columns={'1_Auspraegung_Code' : 'ARS', 
                                      '2_Auspraegung_Label' : 'Alter',
                                      'PRS018__PO0006' : 'Personen_Prozent'})
alter_raw.reset_index(inplace=True)
alter_raw = alter_raw.drop(['index'], axis=1)
alter_raw

,ARS,Alter,Personen_Prozent
0,010010000000,Insgesamt,"100,0"
1,010010000000,Unter 3 Jahre,"2,3"
2,010010000000,3 bis 5 Jahre,"2,2"
3,010010000000,6 bis 9 Jahre,"3,2"
4,010010000000,10 bis 15 Jahre,"5,4"
...,...,...,...
136063,160775050039,25 bis 39 Jahre,"13,8"
136064,160775050039,40 bis 59 Jahre,"37,0"
136065,160775050039,60 bis 66 Jahre,"7,9"
136066,160775050039,67 bis 74 Jahre,"10,5"


In [86]:
#alter_raw.info()

In [87]:
# alle Zeilen mit Gesamtwerten / 100% löschen
alter_raw = alter_raw[~((alter_raw['Alter'] == 'Insgesamt') & (alter_raw['Personen_Prozent'] == '100,0'))]
alter_raw

,ARS,Alter,Personen_Prozent
1,010010000000,Unter 3 Jahre,"2,3"
2,010010000000,3 bis 5 Jahre,"2,2"
3,010010000000,6 bis 9 Jahre,"3,2"
4,010010000000,10 bis 15 Jahre,"5,4"
5,010010000000,16 bis 18 Jahre,"2,6"
...,...,...,...
136063,160775050039,25 bis 39 Jahre,"13,8"
136064,160775050039,40 bis 59 Jahre,"37,0"
136065,160775050039,60 bis 66 Jahre,"7,9"
136066,160775050039,67 bis 74 Jahre,"10,5"


In [88]:
# Personen_Prozent als Float umformatieren
alter_raw.loc[:, 'Personen_Prozent'] = alter_raw['Personen_Prozent'].str.replace(',', '.')
alter_raw.loc[:, 'Personen_Prozent'] = alter_raw['Personen_Prozent'].str.replace('-', '0')
alter_raw.loc[:, 'Personen_Prozent'] = alter_raw['Personen_Prozent'].astype(float) 
alter_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124729 entries, 1 to 136067
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   ARS               124729 non-null  object
 1   Alter             124729 non-null  object
 2   Personen_Prozent  124729 non-null  object
dtypes: object(3)
memory usage: 3.8+ MB


In [89]:
#alter_raw

### AGS aus ALS erstellen

In [90]:
alter_raw.loc[:, 'AGS-8'] = alter_raw['ARS'].str[:5] + alter_raw['ARS'].str[9:]
alter_raw = alter_raw.drop(['ARS'], axis=1)
alter_raw

C:\Users\Conny\AppData\Local\Temp\ipykernel_8880\4122951477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alter_raw.loc[:, 'AGS-8'] = alter_raw['ARS'].str[:5] + alter_raw['ARS'].str[9:]


,Alter,Personen_Prozent,AGS-8
1,Unter 3 Jahre,2.3,01001000
2,3 bis 5 Jahre,2.2,01001000
3,6 bis 9 Jahre,3.2,01001000
4,10 bis 15 Jahre,5.4,01001000
5,16 bis 18 Jahre,2.6,01001000
...,...,...,...
136063,25 bis 39 Jahre,13.8,16077039
136064,40 bis 59 Jahre,37.0,16077039
136065,60 bis 66 Jahre,7.9,16077039
136066,67 bis 74 Jahre,10.5,16077039


In [91]:
alter_raw = alter_raw.groupby(['Alter', 'AGS-8'])['Personen_Prozent'].sum().reset_index()
alter_raw

,Alter,AGS-8,Personen_Prozent
0,10 bis 15 Jahre,01001000,5.4
1,10 bis 15 Jahre,01002000,4.7
2,10 bis 15 Jahre,01003000,5.2
3,10 bis 15 Jahre,01004000,6.1
4,10 bis 15 Jahre,01051001,7.8
...,...,...,...
124724,Unter 3 Jahre,16077049,4.0
124725,Unter 3 Jahre,16077051,1.0
124726,Unter 3 Jahre,16077052,2.0
124727,Unter 3 Jahre,16077055,1.9


### Pivotieren

In [92]:
# Umformatieren der Tabelle durch pivotieren
alter = alter_raw.pivot(index='AGS-8', columns='Alter', values='Personen_Prozent')
alter = alter.reset_index(drop=False)
alter

Alter,AGS-8,10 bis 15 Jahre,16 bis 18 Jahre,19 bis 24 Jahre,25 bis 39 Jahre,3 bis 5 Jahre,40 bis 59 Jahre,6 bis 9 Jahre,60 bis 66 Jahre,67 bis 74 Jahre,75 Jahre und älter,Unter 3 Jahre
0,01001000,5.4,2.6,9.4,20.4,2.2,28.1,3.2,7.6,9.5,9.2,2.3
1,01002000,4.7,2.6,10.3,23.4,2.4,27.4,3.1,7.0,8.6,8.0,2.6
2,01003000,5.2,2.8,7.4,18.6,2.4,29.0,3.2,8.0,10.5,10.5,2.4
3,01004000,6.1,3.3,7.5,16.7,2.5,29.8,3.5,8.1,10.4,9.6,2.5
4,01051001,7.8,4.0,6.3,15.5,2.5,28.0,4.2,8.0,10.4,10.6,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...
11334,16077049,4.3,3.3,5.2,17.9,2.1,38.9,2.4,6.1,8.2,8.5,4.0
11335,16077051,3.9,3.3,6.2,11.1,0.0,42.3,1.3,8.5,11.4,11.4,1.0
11336,16077052,4.0,1.5,5.4,15.2,2.2,37.9,2.9,10.3,10.2,8.6,2.0
11337,16077055,4.1,1.9,5.3,15.5,1.6,34.9,3.1,6.7,14.1,11.0,1.9


### Test Merge

In [93]:
# merge mit Marktstammdaten
marktstd_test = marktstd.merge(alter, on="AGS-8", how="left")
#marktstd_test

In [94]:
marktstd_test.isna().sum()

DatumLetzteAktualisierung                       0
NetzbetreiberpruefungStatus                     0
AnlagenbetreiberMastrNummer                     0
Bundesland                                      0
Landkreis                                       0
Gemeinde                                        0
AGS-8                                           0
Postleitzahl                                    0
Ort                                             0
Registrierungsdatum                             0
Inbetriebnahmedatum                             0
EinheitBetriebsstatus                           0
Bruttoleistung                                  0
FernsteuerbarkeitNb                             0
Einspeisungsart                                 0
ZugeordneteWirkleistungWechselrichter           0
GemeinsamerWechselrichterMitSpeicher            0
AnzahlModule                                    0
Lage                                            0
Leistungsbegrenzung                             0


In [95]:
# fehlende Werte mit denen des am nächsten verfügbaren AGS 8-stellig ersetzen
marktstd_test = marktstd_test.sort_values(by='AGS-8')
marktstd_filled = marktstd_test.fillna(method='ffill').fillna(method='bfill')
marktstd_filled.isna().sum()
# Quelle: ChatGPT (siehe ChatGPT Protokoll, Seite 4)

DatumLetzteAktualisierung                   0
NetzbetreiberpruefungStatus                 0
AnlagenbetreiberMastrNummer                 0
Bundesland                                  0
Landkreis                                   0
Gemeinde                                    0
AGS-8                                       0
Postleitzahl                                0
Ort                                         0
Registrierungsdatum                         0
Inbetriebnahmedatum                         0
EinheitBetriebsstatus                       0
Bruttoleistung                              0
FernsteuerbarkeitNb                         0
Einspeisungsart                             0
ZugeordneteWirkleistungWechselrichter       0
GemeinsamerWechselrichterMitSpeicher        0
AnzahlModule                                0
Lage                                        0
Leistungsbegrenzung                         0
EinheitlicheAusrichtungUndNeigungswinkel    0
Hauptausrichtung                  

In [96]:
alter = marktstd_filled[['AGS-8', 'Unter 3 Jahre', '3 bis 5 Jahre', '6 bis 9 Jahre',
                        '10 bis 15 Jahre', '16 bis 18 Jahre', '19 bis 24 Jahre', '25 bis 39 Jahre',
                        '40 bis 59 Jahre', '60 bis 66 Jahre', '67 bis 74 Jahre', '75 Jahre und älter']].copy()
alter

,AGS-8,Unter 3 Jahre,3 bis 5 Jahre,6 bis 9 Jahre,10 bis 15 Jahre,16 bis 18 Jahre,19 bis 24 Jahre,25 bis 39 Jahre,40 bis 59 Jahre,60 bis 66 Jahre,67 bis 74 Jahre,75 Jahre und älter
2465490,01001000,2.3,2.2,3.2,5.4,2.6,9.4,20.4,28.1,7.6,9.5,9.2
1990778,01001000,2.3,2.2,3.2,5.4,2.6,9.4,20.4,28.1,7.6,9.5,9.2
2703704,01001000,2.3,2.2,3.2,5.4,2.6,9.4,20.4,28.1,7.6,9.5,9.2
647823,01001000,2.3,2.2,3.2,5.4,2.6,9.4,20.4,28.1,7.6,9.5,9.2
1990678,01001000,2.3,2.2,3.2,5.4,2.6,9.4,20.4,28.1,7.6,9.5,9.2
...,...,...,...,...,...,...,...,...,...,...,...,...
74098,16077052,2.0,2.2,2.9,4.0,1.5,5.4,15.2,37.9,10.3,10.2,8.6
820566,16077052,2.0,2.2,2.9,4.0,1.5,5.4,15.2,37.9,10.3,10.2,8.6
1624017,16077052,2.0,2.2,2.9,4.0,1.5,5.4,15.2,37.9,10.3,10.2,8.6
2574378,16077052,2.0,2.2,2.9,4.0,1.5,5.4,15.2,37.9,10.3,10.2,8.6


In [97]:
alter = alter.drop_duplicates(subset=['AGS-8']).reset_index(drop=True)
alter

,AGS-8,Unter 3 Jahre,3 bis 5 Jahre,6 bis 9 Jahre,10 bis 15 Jahre,16 bis 18 Jahre,19 bis 24 Jahre,25 bis 39 Jahre,40 bis 59 Jahre,60 bis 66 Jahre,67 bis 74 Jahre,75 Jahre und älter
0,01001000,2.3,2.2,3.2,5.4,2.6,9.4,20.4,28.1,7.6,9.5,9.2
1,01002000,2.6,2.4,3.1,4.7,2.6,10.3,23.4,27.4,7.0,8.6,8.0
2,01003000,2.4,2.4,3.2,5.2,2.8,7.4,18.6,29.0,8.0,10.5,10.5
3,01004000,2.5,2.5,3.5,6.1,3.3,7.5,16.7,29.8,8.1,10.4,9.6
4,01051001,2.6,2.5,4.2,7.8,4.0,6.3,15.5,28.0,8.0,10.4,10.6
...,...,...,...,...,...,...,...,...,...,...,...,...
11102,16077048,1.3,2.0,3.3,3.5,1.2,4.8,13.9,33.7,8.9,12.1,15.0
11103,16077049,4.0,2.1,2.4,4.3,3.3,5.2,17.9,38.9,6.1,8.2,8.5
11104,16077051,1.0,0.0,1.3,3.9,3.3,6.2,11.1,42.3,8.5,11.4,11.4
11105,16077052,2.0,2.2,2.9,4.0,1.5,5.4,15.2,37.9,10.3,10.2,8.6


## Migrationshintergrund
Berichtszeitpunkt: 09.05.2011    
Tabelle:  2000S-1010, Personen: Migrationshintergrund und -erfahrung   
Quelle: https://ergebnisse2011.zensus2022.de/datenbank/online/statistic/2000S/table/2000S-1010   

**Definition**   
Dieses Merkmal gibt an, ob eine Person einen Migrationshintergrund ausweist
oder nicht.
Als Personen mit Migrationshintergrund werden alle zugewanderten und nicht
zugewanderten Ausländer/-innen sowie alle nach 1955 auf das heutige Gebiet
der Bundesrepublik Deutschland zugewanderten Deutschen und alle Deutschen
mit zumindest einem nach 1955 auf das heutige Gebiet der Bundesrepublik
Deutschland zugewanderten Elternteil definiert.
Bei Vorliegen eines Migrationshintergrunds wird dieser detaillierter klassifiziert.
„Ausländer/-innen“ sind Personen, die nicht die deutsche Staatsangehörigkeit
besitzen (inkl. Personen die staatenlos sind oder deren Staatsangehörigkeit
ungeklärt oder ohne Angabe ist).   
„Deutsche“ sind Personen, die mindestens die deutsche Staatsangehörigkeit
besitzen.  
„Personen mit eigener Migrationserfahrung“ sind Personen, die nicht in
Deutschland geboren und damit nach Deutschland zugezogen sind.   
„Personen ohne eigene Migrationserfahrung“ sind Personen, die in Deutschland
geboren und damit nicht nach Deutschland zugezogen sind.    
„Personen mit beidseitigem Migrationshintergrund“ sind Personen, deren beiden
Elternteile nach 1955 auf das Gebiet der Bundesrepublik Deutschland
zugewandert sind.   
„Personen mit einseitigem Migrationshintergrund“ sind Personen mit einem nach
1955 auf das Gebiet der Bundesrepublik Deutschland zugewanderten Elternteil.    
Statistische Einheit: Person    

In [98]:
migration_raw = pd.read_csv("../input/Data/2000S-1010_flat.csv" ,encoding="latin1", 
                            dtype={"1_Auspraegung_Code" : str}, sep=";", low_memory=False)
migration_raw

,Statistik_Code,Statistik_Label,Zeit_Code,Zeit_Label,Zeit,1_Merkmal_Code,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Code,2_Merkmal_Label,2_Auspraegung_Code,2_Auspraegung_Label,PRS004__Personen__Anzahl
0,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,09.05.2011,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,010010000000,"Flensburg, Stadt",MIGAT1,Migrationshintergrund und -erfahrung,NaN,Insgesamt,81780
1,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,09.05.2011,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,010010000000,"Flensburg, Stadt",MIGAT1,Migrationshintergrund und -erfahrung,MIGH-PERS-X,Personen ohne Migrationshintergrund,68730
2,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,09.05.2011,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,010010000000,"Flensburg, Stadt",MIGAT1,Migrationshintergrund und -erfahrung,MIGH-PERS,Personen mit Migrationshintergrund,13050
3,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,09.05.2011,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,010010000000,"Flensburg, Stadt",MIGAT1,Migrationshintergrund und -erfahrung,MIGE-AUSL-01,Ausländer/-innen,5110
4,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,09.05.2011,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,010010000000,"Flensburg, Stadt",MIGAT1,Migrationshintergrund und -erfahrung,MIGE-AUSL,Ausländer/-innen mit eigener Migrationserf...,4290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17309,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,09.05.2011,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,160770043043,"Schmölln, Stadt",MIGAT1,Migrationshintergrund und -erfahrung,MIGH-DEUT,Deutsche mit Migrationshintergrund,280
17310,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,09.05.2011,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,160770043043,"Schmölln, Stadt",MIGAT1,Migrationshintergrund und -erfahrung,MIGE-DEUT,Deutsche mit eigener Migrationserfahrung,200
17311,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,09.05.2011,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,160770043043,"Schmölln, Stadt",MIGAT1,Migrationshintergrund und -erfahrung,MIGE-DEUT-X,Deutsche ohne eigene Migrationserfahrung,/
17312,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,09.05.2011,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,160770043043,"Schmölln, Stadt",MIGAT1,Migrationshintergrund und -erfahrung,MIGH-DEUT-02,Deutsche mit beidseitigem Migrationshint...,/


### Data Cleaning

In [99]:
# Spalten löschen
drop_cols = ['Statistik_Code', 'Statistik_Label', 'Zeit_Code', 'Zeit_Label', 'Zeit', '2_Merkmal_Code',
                 '1_Merkmal_Code', '2_Merkmal_Label', '2_Auspraegung_Code', '1_Merkmal_Label']
migration_raw = migration_raw.drop(drop_cols, axis=1)
migration_raw.head(10)

,1_Auspraegung_Code,1_Auspraegung_Label,2_Auspraegung_Label,PRS004__Personen__Anzahl
0,010010000000,"Flensburg, Stadt",Insgesamt,81780
1,010010000000,"Flensburg, Stadt",Personen ohne Migrationshintergrund,68730
2,010010000000,"Flensburg, Stadt",Personen mit Migrationshintergrund,13050
3,010010000000,"Flensburg, Stadt",Ausländer/-innen,5110
4,010010000000,"Flensburg, Stadt",Ausländer/-innen mit eigener Migrationserf...,4290
5,010010000000,"Flensburg, Stadt",Ausländer/-innen ohne eigene Migrationserf...,820
6,010010000000,"Flensburg, Stadt",Deutsche mit Migrationshintergrund,7940
7,010010000000,"Flensburg, Stadt",Deutsche mit eigener Migrationserfahrung,4340
8,010010000000,"Flensburg, Stadt",Deutsche ohne eigene Migrationserfahrung,3600
9,010010000000,"Flensburg, Stadt",Deutsche mit beidseitigem Migrationshint...,1730


In [100]:
#migration_raw.isna().sum()

In [101]:
# Spalten umbenennen 
migration_raw = migration_raw.rename(columns={'1_Auspraegung_Code' : 'ARS', '2_Auspraegung_Label': 'Status', 'PRS004__Personen__Anzahl': 'Personen_Anzahl'})
migration_raw.reset_index(inplace=True)
migration_raw = migration_raw.drop(['index'], axis=1)
# migration_raw.head(11)

In [102]:
# migration_raw['Personen_Anzahl'].isin(['-']).sum()

In [103]:
migration_raw['Personen_Anzahl'] = migration_raw['Personen_Anzahl'].astype(str) 
migration_raw['Personen_Anzahl'] = migration_raw['Personen_Anzahl'].replace('/', np.nan)
migration_raw['Personen_Anzahl'] = migration_raw['Personen_Anzahl'].replace('-', np.nan)
migration_raw['Personen_Anzahl'] = migration_raw['Personen_Anzahl'].astype(float) 
migration_raw

,ARS,1_Auspraegung_Label,Status,Personen_Anzahl
0,010010000000,"Flensburg, Stadt",Insgesamt,81780.0
1,010010000000,"Flensburg, Stadt",Personen ohne Migrationshintergrund,68730.0
2,010010000000,"Flensburg, Stadt",Personen mit Migrationshintergrund,13050.0
3,010010000000,"Flensburg, Stadt",Ausländer/-innen,5110.0
4,010010000000,"Flensburg, Stadt",Ausländer/-innen mit eigener Migrationserf...,4290.0
...,...,...,...,...
17309,160770043043,"Schmölln, Stadt",Deutsche mit Migrationshintergrund,280.0
17310,160770043043,"Schmölln, Stadt",Deutsche mit eigener Migrationserfahrung,200.0
17311,160770043043,"Schmölln, Stadt",Deutsche ohne eigene Migrationserfahrung,NaN
17312,160770043043,"Schmölln, Stadt",Deutsche mit beidseitigem Migrationshint...,NaN


### AGS aus ARS erstellen

In [104]:
migration_raw['ARS'] = migration_raw['ARS'].astype(str) 
migration_raw['AGS-8'] = migration_raw['ARS'].str[:5] + migration_raw['ARS'].str[9:]
migration_raw = migration_raw.drop(['ARS', '1_Auspraegung_Label'], axis=1)
#migration_raw

In [105]:
migration_raw = migration_raw.groupby(['Status', 'AGS-8'])['Personen_Anzahl'].sum().reset_index()
migration_raw

,Status,AGS-8,Personen_Anzahl
0,Deutsche mit beidseitigem Migrationshint...,01001000,1730.0
1,Deutsche mit beidseitigem Migrationshint...,01002000,7230.0
2,Deutsche mit beidseitigem Migrationshint...,01003000,4300.0
3,Deutsche mit beidseitigem Migrationshint...,01004000,1950.0
4,Deutsche mit beidseitigem Migrationshint...,01051011,260.0
...,...,...,...
17309,Personen ohne Migrationshintergrund,16076022,20390.0
17310,Personen ohne Migrationshintergrund,16076087,14670.0
17311,Personen ohne Migrationshintergrund,16077001,33190.0
17312,Personen ohne Migrationshintergrund,16077032,10730.0


### Pivotieren

In [106]:
# Umformatieren der Tabelle durch pivotieren
migration = migration_raw.pivot(index='AGS-8', columns='Status', values='Personen_Anzahl')
migration = migration.sort_values(by='AGS-8')
migration = migration.reset_index()
migration

Status,AGS-8,Deutsche mit beidseitigem Migrationshintergrund,Deutsche mit einseitigem Migrationshintergrund,Ausländer/-innen mit eigener Migrationserfahrung,Ausländer/-innen ohne eigene Migrationserfahrung,Deutsche mit eigener Migrationserfahrung,Deutsche ohne eigene Migrationserfahrung,Ausländer/-innen,Deutsche mit Migrationshintergrund,Insgesamt,Personen mit Migrationshintergrund,Personen ohne Migrationshintergrund
0,01001000,1730.0,1860.0,4290.0,820.0,4340.0,3600.0,5110.0,7940.0,81780.0,13050.0,68730.0
1,01002000,7230.0,5960.0,13100.0,2850.0,15200.0,13190.0,15950.0,28380.0,234550.0,44330.0,190220.0
2,01003000,4300.0,5140.0,10110.0,2630.0,12780.0,9450.0,12740.0,22230.0,208000.0,34970.0,173030.0
3,01004000,1950.0,1810.0,3630.0,740.0,4780.0,3760.0,4370.0,8540.0,76280.0,12910.0,63370.0
4,01051011,260.0,220.0,360.0,0.0,840.0,480.0,430.0,1320.0,12790.0,1750.0,11040.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1569,16076022,0.0,0.0,0.0,0.0,120.0,0.0,260.0,190.0,20830.0,440.0,20390.0
1570,16076087,0.0,0.0,170.0,0.0,250.0,0.0,200.0,350.0,15220.0,540.0,14670.0
1571,16077001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,310.0,33860.0,670.0,33190.0
1572,16077032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,10960.0,230.0,10730.0


In [107]:
# Berechnen der prozentualen Verteilung
ags_8_stellig = migration['AGS-8']
migration_prozent = migration.drop(columns=['AGS-8'])
migration_prozent = migration_prozent.div(migration_prozent['Insgesamt'], axis=0) * 100
migration_prozent.columns = [col + "_Prozent" for col in migration_prozent.columns]
migration_prozent['AGS-8'] = ags_8_stellig
migration_prozent
# Quelle: ChatGPT (siehe ChatGPT Protokoll, Seite 5)

,Deutsche mit beidseitigem Migrationshintergrund_Prozent,Deutsche mit einseitigem Migrationshintergrund_Prozent,Ausländer/-innen mit eigener Migrationserfahrung_Prozent,Ausländer/-innen ohne eigene Migrationserfahrung_Prozent,Deutsche mit eigener Migrationserfahrung_Prozent,Deutsche ohne eigene Migrationserfahrung_Prozent,Ausländer/-innen_Prozent,Deutsche mit Migrationshintergrund_Prozent,Insgesamt_Prozent,Personen mit Migrationshintergrund_Prozent,Personen ohne Migrationshintergrund_Prozent,AGS-8
0,2.115432,2.274395,5.245781,1.002690,5.306921,4.402054,6.248472,9.708975,100.0,15.957447,84.042553,01001000
1,3.082498,2.541036,5.585163,1.215093,6.480495,5.623534,6.800256,12.099766,100.0,18.900021,81.099979,01002000
2,2.067308,2.471154,4.860577,1.264423,6.144231,4.543269,6.125000,10.687500,100.0,16.812500,83.187500,01003000
3,2.556371,2.372837,4.758783,0.970110,6.266387,4.929208,5.728894,11.195595,100.0,16.924489,83.075511,01004000
4,2.032838,1.720094,2.814699,0.000000,6.567631,3.752932,3.362002,10.320563,100.0,13.682565,86.317435,01051011
...,...,...,...,...,...,...,...,...,...,...,...,...
1569,0.000000,0.000000,0.000000,0.000000,0.576092,0.000000,1.248200,0.912146,100.0,2.112338,97.887662,16076022
1570,0.000000,0.000000,1.116951,0.000000,1.642576,0.000000,1.314060,2.299606,100.0,3.547963,96.386334,16076087
1571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.915535,100.0,1.978736,98.021264,16077001
1572,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.824818,100.0,2.098540,97.901460,16077032


In [108]:
# nicht benötigte Spalte löschen
migration_prozent = migration_prozent.drop(['Insgesamt_Prozent'], axis=1)
#migration_prozent

In [109]:
# Werte runden
ags_8_stellig = migration_prozent['AGS-8']
migration_prozent = migration_prozent.drop(columns=['AGS-8'])
migration_prozent = migration_prozent.round(2)
migration_prozent['AGS-8'] = ags_8_stellig
migration_prozent

,Deutsche mit beidseitigem Migrationshintergrund_Prozent,Deutsche mit einseitigem Migrationshintergrund_Prozent,Ausländer/-innen mit eigener Migrationserfahrung_Prozent,Ausländer/-innen ohne eigene Migrationserfahrung_Prozent,Deutsche mit eigener Migrationserfahrung_Prozent,Deutsche ohne eigene Migrationserfahrung_Prozent,Ausländer/-innen_Prozent,Deutsche mit Migrationshintergrund_Prozent,Personen mit Migrationshintergrund_Prozent,Personen ohne Migrationshintergrund_Prozent,AGS-8
0,2.12,2.27,5.25,1.00,5.31,4.40,6.25,9.71,15.96,84.04,01001000
1,3.08,2.54,5.59,1.22,6.48,5.62,6.80,12.10,18.90,81.10,01002000
2,2.07,2.47,4.86,1.26,6.14,4.54,6.12,10.69,16.81,83.19,01003000
3,2.56,2.37,4.76,0.97,6.27,4.93,5.73,11.20,16.92,83.08,01004000
4,2.03,1.72,2.81,0.00,6.57,3.75,3.36,10.32,13.68,86.32,01051011
...,...,...,...,...,...,...,...,...,...,...,...
1569,0.00,0.00,0.00,0.00,0.58,0.00,1.25,0.91,2.11,97.89,16076022
1570,0.00,0.00,1.12,0.00,1.64,0.00,1.31,2.30,3.55,96.39,16076087
1571,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.92,1.98,98.02,16077001
1572,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.82,2.10,97.90,16077032


### Test Merge

In [110]:
# merge mit Marktstammdaten
marktstd_test = marktstd.merge(migration_prozent, on="AGS-8", how="left")
marktstd_test

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,...,Deutsche mit beidseitigem Migrationshintergrund_Prozent,Deutsche mit einseitigem Migrationshintergrund_Prozent,Ausländer/-innen mit eigener Migrationserfahrung_Prozent,Ausländer/-innen ohne eigene Migrationserfahrung_Prozent,Deutsche mit eigener Migrationserfahrung_Prozent,Deutsche ohne eigene Migrationserfahrung_Prozent,Ausländer/-innen_Prozent,Deutsche mit Migrationshintergrund_Prozent,Personen mit Migrationshintergrund_Prozent,Personen ohne Migrationshintergrund_Prozent
0,2020-02-20 16:28:35.250812200,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01,...,2.46,2.83,5.66,1.49,7.96,5.29,7.15,13.25,20.40,79.60
1,2021-07-15 07:54:37.863709500,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,...,6.38,2.66,9.27,2.86,13.90,9.06,12.13,22.94,35.09,64.93
2,2019-05-28 08:26:38.031669200,1,ABR983885950021,Brandenburg,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,...,0.00,0.00,0.98,0.00,1.41,0.92,1.53,2.33,3.87,96.13
3,2020-12-07 08:19:07.986079300,1,ABR980052996196,Bayern,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-25 11:18:30.043360300,1,ABR970373690446,Saarland,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,...,3.07,2.43,6.14,2.43,9.71,5.50,8.57,15.21,23.78,76.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173005,2020-07-14 07:21:05.450956000,1,ABR941126007873,Nordrhein-Westfalen,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,...,1.92,1.85,2.70,1.14,4.69,3.77,3.84,8.46,12.37,87.63
3173006,2020-04-29 09:37:03.402485600,0,ABR903375397990,Nordrhein-Westfalen,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,...,3.62,2.91,5.91,3.62,5.29,6.53,9.52,11.82,21.43,78.66
3173007,2020-11-26 08:57:11.006049400,1,ABR985070432688,Bayern,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3173008,2020-05-02 10:16:32.781845900,1,ABR925833676461,Hessen,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,...,2.76,1.78,5.57,2.20,6.18,4.54,7.81,10.72,18.53,81.47


In [111]:
# Duplikate 8-stellig löschen
marktstd_test = marktstd_test.drop_duplicates(subset='AGS-8', keep='first')

In [112]:
# Fehlende Werte
marktstd_test.isna().sum()

DatumLetzteAktualisierung                                           0
NetzbetreiberpruefungStatus                                         0
AnlagenbetreiberMastrNummer                                         0
Bundesland                                                          0
Landkreis                                                           0
Gemeinde                                                            0
AGS-8                                                               0
Postleitzahl                                                        0
Ort                                                                 0
Registrierungsdatum                                                 0
Inbetriebnahmedatum                                                 0
EinheitBetriebsstatus                                               0
Bruttoleistung                                                      0
FernsteuerbarkeitNb                                                 0
Einspeisungsart     

In [113]:
# für 9567 Gemeindeschlüssel liegen keine Daten zum Migrationshintergrund vor
# fehlende Werte mit denen des am nächsten verfügbaren AGS 8-stellig ersetzen
marktstd_test = marktstd_test.sort_values(by='AGS-8')
marktstd_filled = marktstd_test.fillna(method='ffill').fillna(method='bfill')
# marktstd_filled.isna().sum()
# Quelle: ChatGPT (siehe ChatGPT Protokoll, Seite 4)

In [114]:
# Tabelle Migrationshintergrund
selected_columns = ['AGS-8'] + list(marktstd_filled.columns[-10:])
migration = marktstd_filled[selected_columns].reset_index()
migration = migration.drop(['index'], axis=1)
migration

,AGS-8,Deutsche mit beidseitigem Migrationshintergrund_Prozent,Deutsche mit einseitigem Migrationshintergrund_Prozent,Ausländer/-innen mit eigener Migrationserfahrung_Prozent,Ausländer/-innen ohne eigene Migrationserfahrung_Prozent,Deutsche mit eigener Migrationserfahrung_Prozent,Deutsche ohne eigene Migrationserfahrung_Prozent,Ausländer/-innen_Prozent,Deutsche mit Migrationshintergrund_Prozent,Personen mit Migrationshintergrund_Prozent,Personen ohne Migrationshintergrund_Prozent
0,01001000,2.12,2.27,5.25,1.00,5.31,4.40,6.25,9.71,15.96,84.04
1,01002000,3.08,2.54,5.59,1.22,6.48,5.62,6.80,12.10,18.90,81.10
2,01003000,2.07,2.47,4.86,1.26,6.14,4.54,6.12,10.69,16.81,83.19
3,01004000,2.56,2.37,4.76,0.97,6.27,4.93,5.73,11.20,16.92,83.08
4,01051001,2.56,2.37,4.76,0.97,6.27,4.93,5.73,11.20,16.92,83.08
...,...,...,...,...,...,...,...,...,...,...,...
11102,16077048,0.00,0.00,1.04,0.00,1.73,0.00,1.29,2.42,3.71,96.29
11103,16077049,0.00,0.00,1.04,0.00,1.73,0.00,1.29,2.42,3.71,96.29
11104,16077051,0.00,0.00,1.04,0.00,1.73,0.00,1.29,2.42,3.71,96.29
11105,16077052,0.00,0.00,1.04,0.00,1.73,0.00,1.29,2.42,3.71,96.29


## Familienstand
Berichtszeitpunkt: 09.05.2011   
Quelle: https://ergebnisse2011.zensus2022.de/datenbank/online/statistic/1000A/table/1000A-3002   
Tabelle 1000A-3001  

In [115]:
person_raw = pd.read_csv("../input/Data/1000A-3002_flat.csv" ,encoding="latin1", sep=";", low_memory=False)
pd.set_option('display.max_columns', None)
person_raw

,Statistik_Code,Statistik_Label,Zeit_Code,Zeit_Label,Zeit,1_Merkmal_Code,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Code,2_Merkmal_Label,2_Auspraegung_Code,2_Auspraegung_Label,3_Merkmal_Code,3_Merkmal_Label,3_Auspraegung_Code,3_Auspraegung_Label,4_Merkmal_Code,4_Merkmal_Label,4_Auspraegung_Code,4_Auspraegung_Label,PRS001__Personen__Anzahl,PRS018__Personen__Anzahl
0,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,LEDIG,Ledig,GESCH1,Geschlecht,NaN,Insgesamt,NaN,37681
1,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,LEDIG,Ledig,GESCH1,Geschlecht,GESM,Männlich,NaN,20782
2,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,LEDIG,Ledig,GESCH1,Geschlecht,GESW,Weiblich,NaN,16900
3,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,VERHEIRATET-01,Verheiratet/Eingetr. Lebenspartnerschaft,GESCH1,Geschlecht,NaN,Insgesamt,NaN,31131
4,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEOGM1,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,VERHEIRATET-01,Verheiratet/Eingetr. Lebenspartnerschaft,GESCH1,Geschlecht,GESM,Männlich,NaN,15581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020595,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEODL1,Deutschland,DG,Deutschland,ALTKL1,Alter (5 Altersklassen),ALT065BXXX,65 Jahre und älter,FAMST2,Familienstand,GESCHIEDEN-01,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,GESCH1,Geschlecht,GESM,Männlich,415130.0,NaN
1020596,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEODL1,Deutschland,DG,Deutschland,ALTKL1,Alter (5 Altersklassen),ALT065BXXX,65 Jahre und älter,FAMST2,Familienstand,GESCHIEDEN-01,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,GESCH1,Geschlecht,GESW,Weiblich,709428.0,NaN
1020597,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEODL1,Deutschland,DG,Deutschland,ALTKL1,Alter (5 Altersklassen),ALT065BXXX,65 Jahre und älter,FAMST2,Familienstand,FAMST-X,Ohne Angabe,GESCH1,Geschlecht,NaN,Insgesamt,1091.0,NaN
1020598,1000A,Bevölkerung kompakt,STAG,Stichtag,09.05.2011,GEODL1,Deutschland,DG,Deutschland,ALTKL1,Alter (5 Altersklassen),ALT065BXXX,65 Jahre und älter,FAMST2,Familienstand,FAMST-X,Ohne Angabe,GESCH1,Geschlecht,GESM,Männlich,428.0,NaN


### Data Cleaning

In [116]:
# Spalten löschen
drop_cols = ['Statistik_Code', 'Statistik_Label', 'Zeit_Code', 'Zeit_Label', 'Zeit', '1_Merkmal_Code']
person_raw = person_raw.drop(drop_cols, axis=1)
person_raw.head()

,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Code,2_Merkmal_Label,2_Auspraegung_Code,2_Auspraegung_Label,3_Merkmal_Code,3_Merkmal_Label,3_Auspraegung_Code,3_Auspraegung_Label,4_Merkmal_Code,4_Merkmal_Label,4_Auspraegung_Code,4_Auspraegung_Label,PRS001__Personen__Anzahl,PRS018__Personen__Anzahl
0,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,LEDIG,Ledig,GESCH1,Geschlecht,NaN,Insgesamt,NaN,37681
1,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,LEDIG,Ledig,GESCH1,Geschlecht,GESM,Männlich,NaN,20782
2,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,LEDIG,Ledig,GESCH1,Geschlecht,GESW,Weiblich,NaN,16900
3,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,VERHEIRATET-01,Verheiratet/Eingetr. Lebenspartnerschaft,GESCH1,Geschlecht,NaN,Insgesamt,NaN,31131
4,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,VERHEIRATET-01,Verheiratet/Eingetr. Lebenspartnerschaft,GESCH1,Geschlecht,GESM,Männlich,NaN,15581


In [117]:
# nur Gemeinden behalten / Daten für gesamt Deutschland löschen
person_raw = person_raw[person_raw['1_Merkmal_Label'].isin(['Gemeinden'])]
person_raw

,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Code,2_Merkmal_Label,2_Auspraegung_Code,2_Auspraegung_Label,3_Merkmal_Code,3_Merkmal_Label,3_Auspraegung_Code,3_Auspraegung_Label,4_Merkmal_Code,4_Merkmal_Label,4_Auspraegung_Code,4_Auspraegung_Label,PRS001__Personen__Anzahl,PRS018__Personen__Anzahl
0,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,LEDIG,Ledig,GESCH1,Geschlecht,NaN,Insgesamt,NaN,37681
1,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,LEDIG,Ledig,GESCH1,Geschlecht,GESM,Männlich,NaN,20782
2,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,LEDIG,Ledig,GESCH1,Geschlecht,GESW,Weiblich,NaN,16900
3,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,VERHEIRATET-01,Verheiratet/Eingetr. Lebenspartnerschaft,GESCH1,Geschlecht,NaN,Insgesamt,NaN,31131
4,Gemeinden,010010000000,"Flensburg, Stadt",ALTKL1,Alter (5 Altersklassen),NaN,Insgesamt,FAMST2,Familienstand,VERHEIRATET-01,Verheiratet/Eingetr. Lebenspartnerschaft,GESCH1,Geschlecht,GESM,Männlich,NaN,15581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020505,Gemeinden,160775050039,Ponitz,ALTKL1,Alter (5 Altersklassen),ALT065BXXX,65 Jahre und älter,FAMST2,Familienstand,GESCHIEDEN-01,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,GESCH1,Geschlecht,GESM,Männlich,NaN,10
1020506,Gemeinden,160775050039,Ponitz,ALTKL1,Alter (5 Altersklassen),ALT065BXXX,65 Jahre und älter,FAMST2,Familienstand,GESCHIEDEN-01,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,GESCH1,Geschlecht,GESW,Weiblich,NaN,16
1020507,Gemeinden,160775050039,Ponitz,ALTKL1,Alter (5 Altersklassen),ALT065BXXX,65 Jahre und älter,FAMST2,Familienstand,FAMST-X,Ohne Angabe,GESCH1,Geschlecht,NaN,Insgesamt,NaN,-
1020508,Gemeinden,160775050039,Ponitz,ALTKL1,Alter (5 Altersklassen),ALT065BXXX,65 Jahre und älter,FAMST2,Familienstand,FAMST-X,Ohne Angabe,GESCH1,Geschlecht,GESM,Männlich,NaN,-


In [118]:
person_raw.isna().sum()

1_Merkmal_Label                   0
1_Auspraegung_Code                0
1_Auspraegung_Label               0
2_Merkmal_Code                    0
2_Merkmal_Label                   0
2_Auspraegung_Code           170085
2_Auspraegung_Label               0
3_Merkmal_Code                    0
3_Merkmal_Label                   0
3_Auspraegung_Code                0
3_Auspraegung_Label               0
4_Merkmal_Code                    0
4_Merkmal_Label                   0
4_Auspraegung_Code           340170
4_Auspraegung_Label               0
PRS001__Personen__Anzahl    1020510
PRS018__Personen__Anzahl          0
dtype: int64

In [119]:
# Aufteilung in männlich/weiblich verwerfen / nur gesamt beibehalten
person_raw = person_raw[~person_raw['4_Auspraegung_Label'].isin(['Männlich', 'Weiblich'])]
# person_raw

Fogende Ausprägungen werden verworfen, da sie zu viele fehlende Werte enthalten oder nicht relevant sind:  
Merkmal 2: Altersklasse   
Merkmal 4: Geschlecht   

In [120]:
# Spalten löschen
drop_cols = ['4_Merkmal_Code', '4_Merkmal_Label', '4_Auspraegung_Code', '4_Auspraegung_Label',
             'PRS001__Personen__Anzahl', '2_Merkmal_Code', '2_Merkmal_Label', '2_Auspraegung_Code', '1_Auspraegung_Label',
              '2_Auspraegung_Label','3_Merkmal_Code', '3_Merkmal_Label', '3_Auspraegung_Code', '1_Merkmal_Label'
            ]
person_raw = person_raw.drop(drop_cols, axis=1)
pd.set_option('display.max_columns', None)
person_raw

,1_Auspraegung_Code,3_Auspraegung_Label,PRS018__Personen__Anzahl
0,010010000000,Ledig,37681
3,010010000000,Verheiratet/Eingetr. Lebenspartnerschaft,31131
6,010010000000,Verwitwet/Eingetr. Lebenspartner/-in verstorben,5900
9,010010000000,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,7541
12,010010000000,Ohne Angabe,6
...,...,...,...
1020495,160775050039,Ledig,4
1020498,160775050039,Verheiratet/Eingetr. Lebenspartnerschaft,228
1020501,160775050039,Verwitwet/Eingetr. Lebenspartner/-in verstorben,99
1020504,160775050039,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,27


In [121]:
# Spalten umbenennen 
person_raw = person_raw.rename(columns={'1_Auspraegung_Code' : 'ARS', '3_Auspraegung_Label': 'Familienstand', 'PRS018__Personen__Anzahl': 'Personen_Anzahl'})
person_raw.reset_index(inplace=True)
person_raw = person_raw.drop(['index'], axis=1)
person_raw

,ARS,Familienstand,Personen_Anzahl
0,010010000000,Ledig,37681
1,010010000000,Verheiratet/Eingetr. Lebenspartnerschaft,31131
2,010010000000,Verwitwet/Eingetr. Lebenspartner/-in verstorben,5900
3,010010000000,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,7541
4,010010000000,Ohne Angabe,6
...,...,...,...
340165,160775050039,Ledig,4
340166,160775050039,Verheiratet/Eingetr. Lebenspartnerschaft,228
340167,160775050039,Verwitwet/Eingetr. Lebenspartner/-in verstorben,99
340168,160775050039,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,27


In [122]:
# Sind die ARS unique?
person_raw['ARS'].nunique() == person_raw['ARS'].count()

False

In [123]:
# person_raw.info()

In [124]:
# Personenanzahl als Float umformatieren
person_raw['Personen_Anzahl'] = person_raw['Personen_Anzahl'].astype(str) 
person_raw['Personen_Anzahl'] = person_raw['Personen_Anzahl'].replace('-', np.nan)
person_raw['Personen_Anzahl'] = person_raw['Personen_Anzahl'].astype(float) 

In [125]:
# Einige ARS kommen mehrfach vor. Diese Werte werden aufaddiert.
person_raw = person_raw.groupby(['Familienstand', 'ARS'])['Personen_Anzahl'].sum().reset_index()
person_raw

,Familienstand,ARS,Personen_Anzahl
0,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010010000000,15078.0
1,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010020000000,42316.0
2,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010030000000,39709.0
3,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010040000000,13877.0
4,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010510011011,1736.0
...,...,...,...
56690,Verwitwet/Eingetr. Lebenspartner/-in verstorben,160775009049,37.0
56691,Verwitwet/Eingetr. Lebenspartner/-in verstorben,160775009051,48.0
56692,Verwitwet/Eingetr. Lebenspartner/-in verstorben,160775050012,838.0
56693,Verwitwet/Eingetr. Lebenspartner/-in verstorben,160775050017,18.0


### AGS aus ARS erstellen

Der 12-stellige **Amtliche Regionalschlüssel (ARS)** wurde 1994 eingeführt und hat den 8-stelligen **Amtlichen Gemeindeschlüssel (AGS)**   
in vielen bereichen ersetzt.   
Der AGS lässt sich nicht in den ARS überführen.   
Aber der ARS kann durch Weglassen des integrierten Verbandsschlüssels (6. bis 9. Stelle) in den AGS umgewandelt werden.    
Da im Markstammdatenregister der AGS benutzt wird, werden die Familienstands-Daten entsprechend angepasst.   

In [126]:
ars_lenght = person_raw['ARS'].apply(len).value_counts()
ars_lenght

ARS
12    56695
Name: count, dtype: int64

In [127]:
person_raw['AGS-8'] = person_raw['ARS'].str[:5] + person_raw['ARS'].str[9:]
person_raw

,Familienstand,ARS,Personen_Anzahl,AGS-8
0,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010010000000,15078.0,01001000
1,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010020000000,42316.0,01002000
2,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010030000000,39709.0,01003000
3,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010040000000,13877.0,01004000
4,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,010510011011,1736.0,01051011
...,...,...,...,...
56690,Verwitwet/Eingetr. Lebenspartner/-in verstorben,160775009049,37.0,16077049
56691,Verwitwet/Eingetr. Lebenspartner/-in verstorben,160775009051,48.0,16077051
56692,Verwitwet/Eingetr. Lebenspartner/-in verstorben,160775050012,838.0,16077012
56693,Verwitwet/Eingetr. Lebenspartner/-in verstorben,160775050017,18.0,16077017


In [128]:
# Spalten formatieren
person_raw['Personen_Anzahl'] = person_raw['Personen_Anzahl'].replace('-', np.nan)
person_raw["AGS-8"] = person_raw["AGS-8"].astype(str)
person_raw["Personen_Anzahl"] = person_raw["Personen_Anzahl"].astype(np.float32)
person_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56695 entries, 0 to 56694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Familienstand    56695 non-null  object 
 1   ARS              56695 non-null  object 
 2   Personen_Anzahl  56695 non-null  float32
 3   AGS-8            56695 non-null  object 
dtypes: float32(1), object(3)
memory usage: 1.5+ MB


In [129]:
person_raw.isna().sum()

Familienstand      0
ARS                0
Personen_Anzahl    0
AGS-8              0
dtype: int64

In [130]:
fam_stand = person_raw.groupby(['Familienstand', 'AGS-8'])['Personen_Anzahl'].sum().reset_index()
fam_stand

,Familienstand,AGS-8,Personen_Anzahl
0,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,01001000,15078.0
1,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,01002000,42316.0
2,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,01003000,39709.0
3,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,01004000,13877.0
4,Geschieden/Eingetr. Lebenspartnerschaft aufgeh...,01051001,533.0
...,...,...,...
56690,Verwitwet/Eingetr. Lebenspartner/-in verstorben,16077049,37.0
56691,Verwitwet/Eingetr. Lebenspartner/-in verstorben,16077051,48.0
56692,Verwitwet/Eingetr. Lebenspartner/-in verstorben,16077052,279.0
56693,Verwitwet/Eingetr. Lebenspartner/-in verstorben,16077055,134.0


### Pivotieren

In [131]:
# Umformatieren der Tabelle durch pivotieren
familienstand = person_raw.pivot(index='AGS-8', columns='Familienstand', values='Personen_Anzahl')
familienstand = familienstand.reset_index()
#familienstand

In [132]:
# Spalten umsortieren
familienstand = familienstand[['AGS-8', 'Ledig', 'Verheiratet/Eingetr. Lebenspartnerschaft',
                              'Geschieden/Eingetr. Lebenspartnerschaft aufgehoben', 'Verwitwet/Eingetr. Lebenspartner/-in verstorben',
                              'Ohne Angabe']]
familienstand

Familienstand,AGS-8,Ledig,Verheiratet/Eingetr. Lebenspartnerschaft,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben,Verwitwet/Eingetr. Lebenspartner/-in verstorben,Ohne Angabe
0,01001000,75366.0,62257.0,15078.0,11799.0,12.0
1,01002000,230795.0,168117.0,42316.0,29018.0,1323.0
2,01003000,177032.0,171955.0,39709.0,31888.0,27.0
3,01004000,61875.0,66491.0,13877.0,12237.0,30.0
4,01051001,2830.0,3102.0,533.0,589.0,24.0
...,...,...,...,...,...,...
11334,16077049,236.0,352.0,32.0,37.0,0.0
11335,16077051,200.0,334.0,36.0,48.0,0.0
11336,16077052,1355.0,2368.0,168.0,279.0,0.0
11337,16077055,576.0,930.0,70.0,134.0,0.0


In [133]:
familienstand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11339 entries, 0 to 11338
Data columns (total 6 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   AGS-8                                               11339 non-null  object 
 1   Ledig                                               11339 non-null  float32
 2   Verheiratet/Eingetr. Lebenspartnerschaft            11339 non-null  float32
 3   Geschieden/Eingetr. Lebenspartnerschaft aufgehoben  11339 non-null  float32
 4   Verwitwet/Eingetr. Lebenspartner/-in verstorben     11339 non-null  float32
 5   Ohne Angabe                                         11339 non-null  float32
dtypes: float32(5), object(1)
memory usage: 310.2+ KB


In [134]:
familienstand_prozent = familienstand.copy()
familienstand_prozent

Familienstand,AGS-8,Ledig,Verheiratet/Eingetr. Lebenspartnerschaft,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben,Verwitwet/Eingetr. Lebenspartner/-in verstorben,Ohne Angabe
0,01001000,75366.0,62257.0,15078.0,11799.0,12.0
1,01002000,230795.0,168117.0,42316.0,29018.0,1323.0
2,01003000,177032.0,171955.0,39709.0,31888.0,27.0
3,01004000,61875.0,66491.0,13877.0,12237.0,30.0
4,01051001,2830.0,3102.0,533.0,589.0,24.0
...,...,...,...,...,...,...
11334,16077049,236.0,352.0,32.0,37.0,0.0
11335,16077051,200.0,334.0,36.0,48.0,0.0
11336,16077052,1355.0,2368.0,168.0,279.0,0.0
11337,16077055,576.0,930.0,70.0,134.0,0.0


In [135]:
# Personen gesamt berechnen
fs_columns = ["Ledig", "Verheiratet/Eingetr. Lebenspartnerschaft", "Geschieden/Eingetr. Lebenspartnerschaft aufgehoben",
           "Verwitwet/Eingetr. Lebenspartner/-in verstorben", "Ohne Angabe"]
familienstand_prozent['Gesamt'] = familienstand_prozent[fs_columns].sum(axis=1)

In [136]:
# relative Werte berechnen
for col in fs_columns:
    familienstand_prozent[col + '_Prozent'] = (familienstand_prozent[col] / familienstand_prozent['Gesamt']) * 100

In [137]:
for col in fs_columns:
    familienstand_prozent[col + '_Prozent'] = familienstand_prozent[col + '_Prozent'].round(2)
    familienstand_prozent['Verheiratet/Eingetr. Lebenspartnerschaft_Prozent'] = familienstand_prozent['Verheiratet/Eingetr. Lebenspartnerschaft_Prozent'].round(2)
familienstand_prozent

Familienstand,AGS-8,Ledig,Verheiratet/Eingetr. Lebenspartnerschaft,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben,Verwitwet/Eingetr. Lebenspartner/-in verstorben,Ohne Angabe,Gesamt,Ledig_Prozent,Verheiratet/Eingetr. Lebenspartnerschaft_Prozent,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben_Prozent,Verwitwet/Eingetr. Lebenspartner/-in verstorben_Prozent,Ohne Angabe_Prozent
0,01001000,75366.0,62257.0,15078.0,11799.0,12.0,164512.0,45.810001,37.840000,9.17,7.17,0.01
1,01002000,230795.0,168117.0,42316.0,29018.0,1323.0,471569.0,48.939999,35.650002,8.97,6.15,0.28
2,01003000,177032.0,171955.0,39709.0,31888.0,27.0,420611.0,42.090000,40.880001,9.44,7.58,0.01
3,01004000,61875.0,66491.0,13877.0,12237.0,30.0,154510.0,40.049999,43.029999,8.98,7.92,0.02
4,01051001,2830.0,3102.0,533.0,589.0,24.0,7078.0,39.980000,43.830002,7.53,8.32,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...
11334,16077049,236.0,352.0,32.0,37.0,0.0,657.0,35.919998,53.580002,4.87,5.63,0.00
11335,16077051,200.0,334.0,36.0,48.0,0.0,618.0,32.360001,54.049999,5.83,7.77,0.00
11336,16077052,1355.0,2368.0,168.0,279.0,0.0,4170.0,32.490002,56.790001,4.03,6.69,0.00
11337,16077055,576.0,930.0,70.0,134.0,0.0,1710.0,33.680000,54.389999,4.09,7.84,0.00


In [138]:
# Spalten löschen
drop_cols = ['Ledig', 'Verheiratet/Eingetr. Lebenspartnerschaft', 'Geschieden/Eingetr. Lebenspartnerschaft aufgehoben', 
             'Verwitwet/Eingetr. Lebenspartner/-in verstorben', 'Ohne Angabe', 'Gesamt']            
familienstand_prozent = familienstand_prozent.drop(drop_cols, axis=1)
familienstand_prozent

Familienstand,AGS-8,Ledig_Prozent,Verheiratet/Eingetr. Lebenspartnerschaft_Prozent,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben_Prozent,Verwitwet/Eingetr. Lebenspartner/-in verstorben_Prozent,Ohne Angabe_Prozent
0,01001000,45.810001,37.840000,9.17,7.17,0.01
1,01002000,48.939999,35.650002,8.97,6.15,0.28
2,01003000,42.090000,40.880001,9.44,7.58,0.01
3,01004000,40.049999,43.029999,8.98,7.92,0.02
4,01051001,39.980000,43.830002,7.53,8.32,0.34
...,...,...,...,...,...,...
11334,16077049,35.919998,53.580002,4.87,5.63,0.00
11335,16077051,32.360001,54.049999,5.83,7.77,0.00
11336,16077052,32.490002,56.790001,4.03,6.69,0.00
11337,16077055,33.680000,54.389999,4.09,7.84,0.00


### Test Merge

In [139]:
# merge mit Marktstammdaten
marktstd_test = marktstd.merge(familienstand_prozent, on="AGS-8", how="left")
marktstd_test

,DatumLetzteAktualisierung,NetzbetreiberpruefungStatus,AnlagenbetreiberMastrNummer,Bundesland,Landkreis,Gemeinde,AGS-8,Postleitzahl,Ort,Registrierungsdatum,Inbetriebnahmedatum,EinheitBetriebsstatus,Bruttoleistung,FernsteuerbarkeitNb,Einspeisungsart,ZugeordneteWirkleistungWechselrichter,GemeinsamerWechselrichterMitSpeicher,AnzahlModule,Lage,Leistungsbegrenzung,EinheitlicheAusrichtungUndNeigungswinkel,Hauptausrichtung,HauptausrichtungNeigungswinkel,Nutzungsbereich,AGS-5,Inbetriebnahmejahr,Ledig_Prozent,Verheiratet/Eingetr. Lebenspartnerschaft_Prozent,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben_Prozent,Verwitwet/Eingetr. Lebenspartner/-in verstorben_Prozent,Ohne Angabe_Prozent
0,2020-02-20 16:28:35.250812200,1,ABR949444220202,Nordrhein-Westfalen,Münster,Münster,05515000,48147,Münster,2019-02-01,2007-07-20,In Betrieb,3.960,0.0,Volleinspeisung,4.00,Kein Stromspeicher vorhanden,32,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,20 - 40 Grad,Haushalt,05515,2007,50.650002,37.810001,6.23,5.31,0.00
1,2021-07-15 07:54:37.863709500,1,ABR930055871044,Baden-Württemberg,Ostalbkreis,Schwäbisch Gmünd,08136065,73529,Schwäbisch Gmünd,2019-01-31,2013-01-31,In Betrieb,7.410,0.0,Teileinspeisung (einschließlich Eigenverbrauch),8.30,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,08136,2013,39.810001,46.509998,6.18,7.49,0.01
2,2019-05-28 08:26:38.031669200,1,ABR983885950021,Brandenburg,Havelland,Nauen,12063208,14641,Nauen,2019-01-31,2016-02-19,In Betrieb,5.040,0.0,Teileinspeisung (einschließlich Eigenverbrauch),5.00,Stromspeicher vorhanden - kein gemeinsamer Wec...,22,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",1.0,Süd,20 - 40 Grad,Haushalt,12063,2016,41.840000,42.450001,7.65,8.06,0.00
3,2020-12-07 08:19:07.986079300,1,ABR980052996196,Bayern,Regensburg,Pentling,09375180,93080,Pentling,2019-01-31,2016-12-16,In Betrieb,6.360,0.0,Teileinspeisung (einschließlich Eigenverbrauch),6.00,Stromspeicher vorhanden - gemeinsamer Wechselr...,27,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, sonstige",1.0,Süd-West,20 - 40 Grad,Haushalt,09375,2016,38.490002,50.410000,5.17,5.92,0.00
4,2021-01-25 11:18:30.043360300,1,ABR970373690446,Saarland,Saarlouis,Saarlouis,10044115,66740,Saarlouis,2019-01-31,2011-12-08,In Betrieb,7.200,0.0,Teileinspeisung (einschließlich Eigenverbrauch),7.60,Kein Stromspeicher vorhanden,42,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,0.0,West,20 - 40 Grad,Haushalt,10044,2011,36.389999,47.150002,8.45,8.00,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173005,2020-07-14 07:21:05.450956000,1,ABR941126007873,Nordrhein-Westfalen,Siegen-Wittgenstein,Bad Laasphe,05970028,57334,Bad Laasphe,2020-04-16,2009-12-18,In Betrieb,5.324,0.0,Volleinspeisung,4.30,Kein Stromspeicher vorhanden,40,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Ost,< 20 Grad,Haushalt,05970,2009,37.029999,50.009998,5.30,7.65,0.00
3173006,2020-04-29 09:37:03.402485600,0,ABR903375397990,Nordrhein-Westfalen,Hochsauerlandkreis,Bestwig,05958008,59909,Bestwig,2020-04-29,2010-07-01,In Betrieb,16.215,0.0,Volleinspeisung,16.50,Kein Stromspeicher vorhanden,98,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)",Nein,1.0,Süd,40 - 60 Grad,Landwirtschaft,05958,2010,40.330002,48.279999,4.48,6.91,0.00
3173007,2020-11-26 08:57:11.006049400,1,ABR985070432688,Bayern,Neustadt a.d. Waldnaab,Bechtsrieth,09374170,92699,Bechtsrieth,2020-04-29,2020-04-28,In Betrieb,29.900,0.0,Teileinspeisung (einschließlich Eigenverbrauch),50.00,Kein Stromspeicher vorhanden,90,"Bauliche Anlagen (Hausdach, Gebäude und Fassade)","Ja, auf 70%",0.0,West,20 - 40 Grad,Haushalt,09374,2020,35.320000,54.759998,4.25,5.67,0.00
3173008,2020-05-02 10:16:32.781845900,1,ABR925833676461,Hessen,Groß-Gerau,Riedstadt,06433011,64560,Riedstadt,2020-04-29,2016-05-31,In Betrieb,3.570,0.0,Teileinspeisung (einschli

In [140]:
# Behalte nur die jeweils erste Zeile einer AGS 8-stellig / Duplikate verwefen
marktstd_test = marktstd_test.drop_duplicates(subset='AGS-8', keep='first')

In [141]:
# Fehlende Werte
marktstd_test.isna().sum()

DatumLetzteAktualisierung                                       0
NetzbetreiberpruefungStatus                                     0
AnlagenbetreiberMastrNummer                                     0
Bundesland                                                      0
Landkreis                                                       0
Gemeinde                                                        0
AGS-8                                                           0
Postleitzahl                                                    0
Ort                                                             0
Registrierungsdatum                                             0
Inbetriebnahmedatum                                             0
EinheitBetriebsstatus                                           0
Bruttoleistung                                                  0
FernsteuerbarkeitNb                                             0
Einspeisungsart                                                 0
Zugeordnet

In [142]:
# fehlende Werte mit denen des am nächsten verfügbaren AGS 8-stellig ersetzen
marktstd_test = marktstd_test.sort_values(by='AGS-8')
marktstd_filled = marktstd_test.fillna(method='ffill').fillna(method='bfill')
marktstd_filled.isna().sum()
# Quelle: ChatGPT (siehe ChatGPT Protokoll, Seite 4)

DatumLetzteAktualisierung                                     0
NetzbetreiberpruefungStatus                                   0
AnlagenbetreiberMastrNummer                                   0
Bundesland                                                    0
Landkreis                                                     0
Gemeinde                                                      0
AGS-8                                                         0
Postleitzahl                                                  0
Ort                                                           0
Registrierungsdatum                                           0
Inbetriebnahmedatum                                           0
EinheitBetriebsstatus                                         0
Bruttoleistung                                                0
FernsteuerbarkeitNb                                           0
Einspeisungsart                                               0
ZugeordneteWirkleistungWechselrichter   

In [143]:
fam_stand = marktstd_filled[['AGS-8', 'Ledig_Prozent', 'Verheiratet/Eingetr. Lebenspartnerschaft_Prozent', 
                        'Geschieden/Eingetr. Lebenspartnerschaft aufgehoben_Prozent', 
             'Verwitwet/Eingetr. Lebenspartner/-in verstorben_Prozent', 'Ohne Angabe_Prozent']].copy()
fam_stand = fam_stand.reset_index()
fam_stand = fam_stand.drop(['index'], axis=1)
fam_stand

,AGS-8,Ledig_Prozent,Verheiratet/Eingetr. Lebenspartnerschaft_Prozent,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben_Prozent,Verwitwet/Eingetr. Lebenspartner/-in verstorben_Prozent,Ohne Angabe_Prozent
0,01001000,45.810001,37.840000,9.17,7.17,0.01
1,01002000,48.939999,35.650002,8.97,6.15,0.28
2,01003000,42.090000,40.880001,9.44,7.58,0.01
3,01004000,40.049999,43.029999,8.98,7.92,0.02
4,01051001,39.980000,43.830002,7.53,8.32,0.34
...,...,...,...,...,...,...
11102,16077048,31.410000,50.160000,7.03,11.41,0.00
11103,16077049,35.919998,53.580002,4.87,5.63,0.00
11104,16077051,32.360001,54.049999,5.83,7.77,0.00
11105,16077052,32.490002,56.790001,4.03,6.69,0.00


In [144]:
fam_stand['Ledig_Prozent'] = fam_stand['Ledig_Prozent'].round(2)
pd.set_option('display.float_format', '{:.2f}'.format)
fam_stand

,AGS-8,Ledig_Prozent,Verheiratet/Eingetr. Lebenspartnerschaft_Prozent,Geschieden/Eingetr. Lebenspartnerschaft aufgehoben_Prozent,Verwitwet/Eingetr. Lebenspartner/-in verstorben_Prozent,Ohne Angabe_Prozent
0,01001000,45.81,37.84,9.17,7.17,0.01
1,01002000,48.94,35.65,8.97,6.15,0.28
2,01003000,42.09,40.88,9.44,7.58,0.01
3,01004000,40.05,43.03,8.98,7.92,0.02
4,01051001,39.98,43.83,7.53,8.32,0.34
...,...,...,...,...,...,...
11102,16077048,31.41,50.16,7.03,11.41,0.00
11103,16077049,35.92,53.58,4.87,5.63,0.00
11104,16077051,32.36,54.05,5.83,7.77,0.00
11105,16077052,32.49,56.79,4.03,6.69,0.00


## Bildungsquoten, Schulabschlüsse
Stand: 30.06.2021
Quelle: https://www.deutschlandatlas.bund.de/DE/Service/Downloads/downloads_node.html
    
**bquali_unifh**: Anteil sozialversicherungspflichtig Beschäftigter am Arbeitsort mit einem 
akademischen Abschluss an allen sozialversicherungspflichtig Beschäftigten im Jahr 2021 in %     

**bquali_mabschl**: Anteil sozialversicherungspflichtig Beschäftigter am Arbeitsort mit   einem anerkannten Berufsabschluss und ohne einen akademischen Abschluss an allen     sozialversicherungspflichtig Beschäftigten im Jahr 2021 in %  

**bquali_oabschl**: Anteil sozialversicherungspflichtig Beschäftigter ohne einen Berufs-/  
akademischen Abschluss an allen sozialversicherungspflichtig Beschäftigten im Jahr 2021 in %      

**schule_oabschl**: Anteil der Schulabgänger/-innen ohne Hauptschulabschluss an allen  
Schulabgänger/-innen allgemeinbildender Schulen im Jahr 2021 in %   
   


In [145]:
# Datensatz einlesen
columns = ['Gebietskennziffer', 'bquali_unifh', 'bquali_mabschl', 
           'bquali_oabschl', 'schule_oabschl']
bildung_raw = pd.read_csv('../input/Deutschlandatlas/Deutschlandatlas_KRS1221.csv', 
                          usecols=columns, encoding="latin1", 
                          dtype={"KRS1221" : str}, sep=";", decimal=",", low_memory=False)

bildung_raw

,Gebietskennziffer,bquali_unifh,bquali_mabschl,bquali_oabschl,schule_oabschl
0,1001000,15.55,62.44,14.18,8.81
1,1002000,21.61,58.24,12.27,8.48
2,1003000,14.48,63.18,13.31,8.28
3,1004000,9.87,65.51,14.41,7.36
4,1051000,8.26,69.09,12.92,9.55
...,...,...,...,...,...
395,16073000,11.77,76.29,7.40,11.57
396,16074000,11.55,75.47,6.95,6.10
397,16075000,8.03,80.23,6.97,8.56
398,16076000,9.69,79.21,5.85,7.38


### Data Cleaning

In [146]:
# Gebietskennziffer umwandeln in Gemeindeschlüssel AGS 5-stellig
bildung_raw['Gebietskennziffer'] = bildung_raw['Gebietskennziffer'].astype(str)
bildung_raw['Gebietskennziffer'] = bildung_raw['Gebietskennziffer'].apply(lambda x: '0' + x if len(x) == 7 else x)
bildung_raw['AGS-5'] = bildung_raw['Gebietskennziffer'].str[:-3]
bildung_raw

,Gebietskennziffer,bquali_unifh,bquali_mabschl,bquali_oabschl,schule_oabschl,AGS-5
0,01001000,15.55,62.44,14.18,8.81,01001
1,01002000,21.61,58.24,12.27,8.48,01002
2,01003000,14.48,63.18,13.31,8.28,01003
3,01004000,9.87,65.51,14.41,7.36,01004
4,01051000,8.26,69.09,12.92,9.55,01051
...,...,...,...,...,...,...
395,16073000,11.77,76.29,7.40,11.57,16073
396,16074000,11.55,75.47,6.95,6.10,16074
397,16075000,8.03,80.23,6.97,8.56,16075
398,16076000,9.69,79.21,5.85,7.38,16076


In [147]:
# nicht benötigte Spalten löschen
bildung_raw = bildung_raw.drop(['Gebietskennziffer'], axis=1)
bildung_raw

,bquali_unifh,bquali_mabschl,bquali_oabschl,schule_oabschl,AGS-5
0,15.55,62.44,14.18,8.81,01001
1,21.61,58.24,12.27,8.48,01002
2,14.48,63.18,13.31,8.28,01003
3,9.87,65.51,14.41,7.36,01004
4,8.26,69.09,12.92,9.55,01051
...,...,...,...,...,...
395,11.77,76.29,7.40,11.57,16073
396,11.55,75.47,6.95,6.10,16074
397,8.03,80.23,6.97,8.56,16075
398,9.69,79.21,5.85,7.38,16076


### Test Merge

In [148]:
# merge mit Marktstammdaten
#marktstd = marktstd.merge(bildung_raw[['bquali_unifh', 'bquali_mabschl', 'bquali_oabschl', 
#                                     'schule_oabschl', 'AGS-5']], on='AGS-5', how='left')
#marktstd

In [149]:
#marktstd.isna().sum()

## Ausländer-Anteil   
Stichtag: 31.12.2021  
Tabelle: Deutschlandatlas_KRS1221.csv
Quelle: https://www.deutschlandatlas.bund.de/DE/Service/Downloads/downloads_node.html

**Auslaender_%**: Anteil der Ausländer/-innen an der Gesamtbevölkerung in 2021 in %

In [150]:
# Datensatz einlesen
columns = ['Gebietskennziffer', 'name', 'bev_ausl']
auslaender_raw = pd.read_csv("../input/Deutschlandatlas/Deutschlandatlas_KRS1221.csv", usecols=columns, encoding="latin1", 
                             dtype={"Gebietskennziffer" : str}, sep=";", decimal=",", low_memory=False)
auslaender_raw = auslaender_raw.rename(columns={'bev_ausl': 'Auslaender_%', 'name' : 'Ort'})
auslaender_raw

,Gebietskennziffer,Ort,Auslaender_%
0,1001000,"Flensburg, Stadt",16.99
1,1002000,"Kiel, Landeshauptstadt",12.50
2,1003000,"Lübeck, Hansestadt",11.21
3,1004000,"Neumünster, Stadt",13.51
4,1051000,Dithmarschen,6.55
...,...,...,...
395,16073000,Saalfeld-Rudolstadt,3.09
396,16074000,Saale-Holzland-Kreis,3.87
397,16075000,Saale-Orla-Kreis,3.97
398,16076000,Greiz,2.99


### Data Cleaning

In [151]:
auslaender_raw.isna().sum()

Gebietskennziffer    0
Ort                  0
Auslaender_%         0
dtype: int64

In [152]:
# Gebietskennziffer umwandeln in Gemeindeschlüssel AGS 5-stellig
auslaender_raw['Gebietskennziffer'] = auslaender_raw['Gebietskennziffer'].apply(lambda x: '0' + x if len(x) == 7 else x)
auslaender_raw['AGS-5'] = auslaender_raw['Gebietskennziffer'].str[:-3]
auslaender_raw

,Gebietskennziffer,Ort,Auslaender_%,AGS-5
0,01001000,"Flensburg, Stadt",16.99,01001
1,01002000,"Kiel, Landeshauptstadt",12.50,01002
2,01003000,"Lübeck, Hansestadt",11.21,01003
3,01004000,"Neumünster, Stadt",13.51,01004
4,01051000,Dithmarschen,6.55,01051
...,...,...,...,...
395,16073000,Saalfeld-Rudolstadt,3.09,16073
396,16074000,Saale-Holzland-Kreis,3.87,16074
397,16075000,Saale-Orla-Kreis,3.97,16075
398,16076000,Greiz,2.99,16076


In [153]:
# nicht benötigte Spalten löschen
auslaender_raw = auslaender_raw.drop(['Gebietskennziffer', 'Ort'], axis=1)
auslaender_raw

,Auslaender_%,AGS-5
0,16.99,01001
1,12.50,01002
2,11.21,01003
3,13.51,01004
4,6.55,01051
...,...,...
395,3.09,16073
396,3.87,16074
397,3.97,16075
398,2.99,16076


### Test Merge

In [154]:
# merge mit Marktstammdaten
#marktstd = marktstd.merge(auslaender_raw, on='AGS-5', how='left')
#marktstd

In [155]:
#marktstd.isna().sum()

## Kriminalität
Gebietsstand zum 31.12.2020   
Tabelle: Deutschlandatlas_KRS1220.csv   
Quelle: https://www.deutschlandatlas.bund.de/DE/Service/Downloads/downloads_node.html   
      
**Straftaten:** Straftaten gesamt pro 100.000 Einwohner in 2020  
**Einbruch:** Wohnungseinbruchdiebstahl pro 100.000 Einwohner in 2020   

In [156]:
# Datensatz einlesen
columns = ["Gebietskennziffer", "name", "straft", "einbr"]
krimi_raw = pd.read_csv("../input/Deutschlandatlas/Deutschlandatlas_KRS1220.csv", usecols=columns, encoding="latin1", 
                             dtype={"Gebietskennziffer" : str}, sep=";", decimal=",", low_memory=False)
krimi_raw = krimi_raw.rename(columns={'straft': 'Straftaten', 'einbr': 'Einbruch', 'name' : 'Ort'})
krimi_raw

,Gebietskennziffer,Ort,Straftaten,Einbruch
0,1001000,"Flensburg, Stadt",9588.14,88.95
1,1002000,"Kiel, Stadt",8340.19,98.54
2,1003000,"Lübeck, Stadt",9222.78,64.86
3,1004000,"Neumünster, Stadt",11094.42,142.67
4,1051000,Dithmarschen,4846.49,63.04
...,...,...,...,...
396,16073000,Saalfeld-Rudolstadt,5567.90,54.83
397,16074000,Saale-Holzland-Kreis,6109.93,35.02
398,16075000,Saale-Orla-Kreis,5276.77,33.91
399,16076000,Greiz,4189.60,22.76


### Data Cleaning

In [157]:
# Gebietskennziffer umwandeln in Gemeindeschlüssel AGS 5-stellig
krimi_raw['Gebietskennziffer'] = krimi_raw['Gebietskennziffer'].apply(lambda x: '0' + x if len(x) == 7 else x)
krimi_raw['AGS-5'] = krimi_raw['Gebietskennziffer'].str[:-3]
krimi_raw

,Gebietskennziffer,Ort,Straftaten,Einbruch,AGS-5
0,01001000,"Flensburg, Stadt",9588.14,88.95,01001
1,01002000,"Kiel, Stadt",8340.19,98.54,01002
2,01003000,"Lübeck, Stadt",9222.78,64.86,01003
3,01004000,"Neumünster, Stadt",11094.42,142.67,01004
4,01051000,Dithmarschen,4846.49,63.04,01051
...,...,...,...,...,...
396,16073000,Saalfeld-Rudolstadt,5567.90,54.83,16073
397,16074000,Saale-Holzland-Kreis,6109.93,35.02,16074
398,16075000,Saale-Orla-Kreis,5276.77,33.91,16075
399,16076000,Greiz,4189.60,22.76,16076


In [158]:
# nicht benötigte Spalten löschen
krimi_raw = krimi_raw.drop(['Gebietskennziffer', 'Ort'], axis=1)
krimi_raw

,Straftaten,Einbruch,AGS-5
0,9588.14,88.95,01001
1,8340.19,98.54,01002
2,9222.78,64.86,01003
3,11094.42,142.67,01004
4,4846.49,63.04,01051
...,...,...,...
396,5567.90,54.83,16073
397,6109.93,35.02,16074
398,5276.77,33.91,16075
399,4189.60,22.76,16076


### Test Merge

In [159]:
# merge mit Marktstammdaten
#marktstd = marktstd.merge(krimi_raw, on='AGS-5', how='left')
#marktstd

In [160]:
#marktstd.isna().sum()

## Bundestagswahlen 2021

Quelle: https://www.bundeswahlleiter.de/bundestagswahlen/2021/ergebnisse/weitere-ergebnisse.html

In [161]:
# Einlesen von angebenenen Spalten und löschen der ersten Zeile
cols_list = ['Land', 'Regierungsbezirk', 'Kreis', 'Gemeinde', 'Gemeinde Name', 'Gültige', 'CDU', 'SPD', 
             'AfD', 'FDP', 'DIE LINKE', 'GRÜNE', 'CSU', 'FREIE WÄHLER', 'Die PARTEI', 'Tierschutzpartei', 
             'NPD', 'PIRATEN']
wahlen_raw = pd.read_csv('../input/Data/btw21_wbz_ergebnisse.csv', sep=';', 
                         dtype={'Land' : str, 'Regierungsbezirk' : str,'Kreis' : str,'Gemeinde' : str,},
                         low_memory=False, usecols=cols_list).drop(0)
wahlen_raw

,Land,Regierungsbezirk,Kreis,Gemeinde,Gemeinde Name,Gültige,CDU,SPD,AfD,FDP,DIE LINKE,GRÜNE,CSU,FREIE WÄHLER,Die PARTEI,Tierschutzpartei,NPD,PIRATEN
1,01,0,01,000,"Flensburg, Stadt",881,129,225,38,48,45,284,0,6,0,0,0,0
2,01,0,01,000,"Flensburg, Stadt",812,114,241,69,48,44,176,0,14,0,0,0,0
3,01,0,01,000,"Flensburg, Stadt",598,76,197,44,34,46,125,0,7,0,0,0,0
4,01,0,01,000,"Flensburg, Stadt",573,54,136,45,22,52,177,0,8,0,0,0,0
5,01,0,01,000,"Flensburg, Stadt",491,27,96,41,23,64,182,0,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94664,10,0,41,519,"Völklingen, Stadt",678,216,248,51,50,30,22,0,23,17,0,0,0
94665,10,0,41,519,"Völklingen, Stadt",790,218,310,66,62,41,33,0,26,13,0,0,0
94666,10,0,41,519,"Völklingen, Stadt",817,176,373,69,85,38,22,0,16,21,0,0,0
94667,10,0,41,519,"Völklingen, Stadt",416,128,154,34,31,26,19,0,13,4,0,0,0


### Gemeindeschlüssel AGS 8-stellig erstellen

**Amtlicher Gemeindeschlüssel (AGS)**   
8-stelliger Schlüssel zur eindeutigen Identifizierung einer Gemeinde mit den Bestandteilen:
Bundesland (2 Stellen), Regierungsbezirk (1 Stelle), Kreis (2 Stellen) und Gemeinde (3 Stellen).

In [162]:
wahlen_raw['AGS-8'] = wahlen_raw.apply(lambda row: row['Land'][:2] + 
                                               row['Regierungsbezirk'][:1] + 
                                               row['Kreis'][:2] + row['Gemeinde'][:3], axis=1)
wahlen_raw

,Land,Regierungsbezirk,Kreis,Gemeinde,Gemeinde Name,Gültige,CDU,SPD,AfD,FDP,DIE LINKE,GRÜNE,CSU,FREIE WÄHLER,Die PARTEI,Tierschutzpartei,NPD,PIRATEN,AGS-8
1,01,0,01,000,"Flensburg, Stadt",881,129,225,38,48,45,284,0,6,0,0,0,0,01001000
2,01,0,01,000,"Flensburg, Stadt",812,114,241,69,48,44,176,0,14,0,0,0,0,01001000
3,01,0,01,000,"Flensburg, Stadt",598,76,197,44,34,46,125,0,7,0,0,0,0,01001000
4,01,0,01,000,"Flensburg, Stadt",573,54,136,45,22,52,177,0,8,0,0,0,0,01001000
5,01,0,01,000,"Flensburg, Stadt",491,27,96,41,23,64,182,0,3,0,0,0,0,01001000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94664,10,0,41,519,"Völklingen, Stadt",678,216,248,51,50,30,22,0,23,17,0,0,0,10041519
94665,10,0,41,519,"Völklingen, Stadt",790,218,310,66,62,41,33,0,26,13,0,0,0,10041519
94666,10,0,41,519,"Völklingen, Stadt",817,176,373,69,85,38,22,0,16,21,0,0,0,10041519
94667,10,0,41,519,"Völklingen, Stadt",416,128,154,34,31,26,19,0,13,4,0,0,0,10041519


### Data Cleaning

In [163]:
# nicht benötigte Spalten löschen
drop_cols = ['Land','Regierungsbezirk','Kreis','Gemeinde','Gemeinde Name',]
wahlen_raw = wahlen_raw.drop(drop_cols, axis=1)
wahlen_raw

,Gültige,CDU,SPD,AfD,FDP,DIE LINKE,GRÜNE,CSU,FREIE WÄHLER,Die PARTEI,Tierschutzpartei,NPD,PIRATEN,AGS-8
1,881,129,225,38,48,45,284,0,6,0,0,0,0,01001000
2,812,114,241,69,48,44,176,0,14,0,0,0,0,01001000
3,598,76,197,44,34,46,125,0,7,0,0,0,0,01001000
4,573,54,136,45,22,52,177,0,8,0,0,0,0,01001000
5,491,27,96,41,23,64,182,0,3,0,0,0,0,01001000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94664,678,216,248,51,50,30,22,0,23,17,0,0,0,10041519
94665,790,218,310,66,62,41,33,0,26,13,0,0,0,10041519
94666,817,176,373,69,85,38,22,0,16,21,0,0,0,10041519
94667,416,128,154,34,31,26,19,0,13,4,0,0,0,10041519


In [164]:
print(wahlen_raw.dtypes)

Gültige             object
CDU                 object
SPD                 object
AfD                 object
FDP                 object
DIE LINKE           object
GRÜNE               object
CSU                 object
FREIE WÄHLER        object
Die PARTEI          object
Tierschutzpartei    object
NPD                 object
PIRATEN             object
AGS-8               object
dtype: object


In [165]:
# Anzahl Stimmen in Integer überführen
wahlen_raw = wahlen_raw.fillna(0)
columns = ['Gültige', 'CDU', 'SPD', 'AfD', 'FDP', 'DIE LINKE', 'GRÜNE', 'CSU', 
                     'FREIE WÄHLER', 'Die PARTEI', 'Tierschutzpartei', 'NPD', 'PIRATEN']
wahlen_raw[columns] = wahlen_raw[columns].astype(int)
wahlen_raw

,Gültige,CDU,SPD,AfD,FDP,DIE LINKE,GRÜNE,CSU,FREIE WÄHLER,Die PARTEI,Tierschutzpartei,NPD,PIRATEN,AGS-8
1,881,129,225,38,48,45,284,0,6,0,0,0,0,01001000
2,812,114,241,69,48,44,176,0,14,0,0,0,0,01001000
3,598,76,197,44,34,46,125,0,7,0,0,0,0,01001000
4,573,54,136,45,22,52,177,0,8,0,0,0,0,01001000
5,491,27,96,41,23,64,182,0,3,0,0,0,0,01001000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94664,678,216,248,51,50,30,22,0,23,17,0,0,0,10041519
94665,790,218,310,66,62,41,33,0,26,13,0,0,0,10041519
94666,817,176,373,69,85,38,22,0,16,21,0,0,0,10041519
94667,416,128,154,34,31,26,19,0,13,4,0,0,0,10041519


In [166]:
# Wahlergebnisse entlang der AGS 8-stellig aggregieren
wahlen_raw = wahlen_raw.groupby('AGS-8').sum().reset_index()
wahlen_raw

,AGS-8,Gültige,CDU,SPD,AfD,FDP,DIE LINKE,GRÜNE,CSU,FREIE WÄHLER,Die PARTEI,Tierschutzpartei,NPD,PIRATEN
0,01001000,50808,7941,10484,2681,3252,2855,18075,0,454,0,0,0,0
1,01002000,139735,24119,41407,6562,10303,6913,40123,0,1013,2766,0,0,0
2,01003000,117356,24960,40376,7551,8341,3709,25882,0,1563,2000,0,0,0
3,01004000,39809,10056,13946,3707,3744,1147,5230,0,559,705,0,0,0
4,01051001,1494,471,409,126,221,59,148,0,33,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11276,16077909,324,67,72,70,49,36,15,0,0,0,0,0,0
11277,16077925,349,60,92,79,26,52,15,0,0,0,0,0,0
11278,16077950,750,180,184,168,75,71,32,0,0,0,0,0,0
11279,16077951,1422,344,257,291,143,131,107,0,0,0,0,0,0


In [167]:
# relative Werte berechnen
prozent_columns = ['Gültige', 'CDU', 'SPD', 'AfD', 'FDP', 'DIE LINKE', 'GRÜNE', 'CSU', 
                     'FREIE WÄHLER', 'Die PARTEI', 'Tierschutzpartei', 'NPD', 'PIRATEN']
for col in prozent_columns:
    wahlen_raw[col + '_Prozent'] = (wahlen_raw[col] / wahlen_raw['Gültige']) * 100
wahlen_raw

,AGS-8,Gültige,CDU,SPD,AfD,FDP,DIE LINKE,GRÜNE,CSU,FREIE WÄHLER,Die PARTEI,Tierschutzpartei,NPD,PIRATEN,Gültige_Prozent,CDU_Prozent,SPD_Prozent,AfD_Prozent,FDP_Prozent,DIE LINKE_Prozent,GRÜNE_Prozent,CSU_Prozent,FREIE WÄHLER_Prozent,Die PARTEI_Prozent,Tierschutzpartei_Prozent,NPD_Prozent,PIRATEN_Prozent
0,01001000,50808,7941,10484,2681,3252,2855,18075,0,454,0,0,0,0,100.00,15.63,20.63,5.28,6.40,5.62,35.58,0.00,0.89,0.00,0.00,0.00,0.00
1,01002000,139735,24119,41407,6562,10303,6913,40123,0,1013,2766,0,0,0,100.00,17.26,29.63,4.70,7.37,4.95,28.71,0.00,0.72,1.98,0.00,0.00,0.00
2,01003000,117356,24960,40376,7551,8341,3709,25882,0,1563,2000,0,0,0,100.00,21.27,34.40,6.43,7.11,3.16,22.05,0.00,1.33,1.70,0.00,0.00,0.00
3,01004000,39809,10056,13946,3707,3744,1147,5230,0,559,705,0,0,0,100.00,25.26,35.03,9.31,9.40,2.88,13.14,0.00,1.40,1.77,0.00,0.00,0.00
4,01051001,1494,471,409,126,221,59,148,0,33,0,0,0,0,100.00,31.53,27.38,8.43,14.79,3.95,9.91,0.00,2.21,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11276,16077909,324,67,72,70,49,36,15,0,0,0,0,0,0,100.00,20.68,22.22,21.60,15.12,11.11,4.63,0.00,0.00,0.00,0.00,0.00,0.00
11277,16077925,349,60,92,79,26,52,15,0,0,0,0,0,0,100.00,17.19,26.36,22.64,7.45,14.90,4.30,0.00,0.00,0.00,0.00,0.00,0.00
11278,16077950,750,180,184,168,75,71,32,0,0,0,0,0,0,100.00,24.00,24.53,22.40,10.00,9.47,4.27,0.00,0.00,0.00,0.00,0.00,0.00
11279,16077951,1422,344,257,291,143,131,107,0,0,0,0,0,0,100.00,24.19,18.07,20.46,10.06,9.21,7.52,0.00,0.00,0.00,0.00,0.00,0.00


In [168]:
# Spalten löschen
drop_cols = ['Gültige', 'CDU', 'SPD', 'AfD', 'FDP', 'DIE LINKE', 'GRÜNE', 'CSU', 
            'FREIE WÄHLER', 'Die PARTEI', 'Tierschutzpartei', 'NPD', 'PIRATEN', 'Gültige_Prozent']           
wahlen_raw = wahlen_raw.drop(drop_cols, axis=1)
wahlen_raw

,AGS-8,CDU_Prozent,SPD_Prozent,AfD_Prozent,FDP_Prozent,DIE LINKE_Prozent,GRÜNE_Prozent,CSU_Prozent,FREIE WÄHLER_Prozent,Die PARTEI_Prozent,Tierschutzpartei_Prozent,NPD_Prozent,PIRATEN_Prozent
0,01001000,15.63,20.63,5.28,6.40,5.62,35.58,0.00,0.89,0.00,0.00,0.00,0.00
1,01002000,17.26,29.63,4.70,7.37,4.95,28.71,0.00,0.72,1.98,0.00,0.00,0.00
2,01003000,21.27,34.40,6.43,7.11,3.16,22.05,0.00,1.33,1.70,0.00,0.00,0.00
3,01004000,25.26,35.03,9.31,9.40,2.88,13.14,0.00,1.40,1.77,0.00,0.00,0.00
4,01051001,31.53,27.38,8.43,14.79,3.95,9.91,0.00,2.21,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11276,16077909,20.68,22.22,21.60,15.12,11.11,4.63,0.00,0.00,0.00,0.00,0.00,0.00
11277,16077925,17.19,26.36,22.64,7.45,14.90,4.30,0.00,0.00,0.00,0.00,0.00,0.00
11278,16077950,24.00,24.53,22.40,10.00,9.47,4.27,0.00,0.00,0.00,0.00,0.00,0.00
11279,16077951,24.19,18.07,20.46,10.06,9.21,7.52,0.00,0.00,0.00,0.00,0.00,0.00


### Test Merge

In [169]:
# merge mit Marktstammdaten
marktstd_test = marktstd.merge(wahlen_raw, on='AGS-8', how='left')
#marktstd_test

In [170]:
marktstd_test = marktstd_test.drop_duplicates(subset='AGS-8', keep='first')

In [171]:
marktstd_test.isna().sum()

DatumLetzteAktualisierung                     0
NetzbetreiberpruefungStatus                   0
AnlagenbetreiberMastrNummer                   0
Bundesland                                    0
Landkreis                                     0
Gemeinde                                      0
AGS-8                                         0
Postleitzahl                                  0
Ort                                           0
Registrierungsdatum                           0
Inbetriebnahmedatum                           0
EinheitBetriebsstatus                         0
Bruttoleistung                                0
FernsteuerbarkeitNb                           0
Einspeisungsart                               0
ZugeordneteWirkleistungWechselrichter         0
GemeinsamerWechselrichterMitSpeicher          0
AnzahlModule                                  0
Lage                                          0
Leistungsbegrenzung                           0
EinheitlicheAusrichtungUndNeigungswinkel

In [172]:
nan = marktstd_test.loc[marktstd_test.drop('AGS-8', axis=1).isna().any(axis=1), 'AGS-8']
nan

267        02000000
351        16068029
424        16070053
564        11000000
660        03154010
             ...   
2898153    16063019
3078866    07232038
3113783    14729230
3142044    16075069
3157189    09572455
Name: AGS-8, Length: 653, dtype: object

In [173]:
# für 653 Gemeindeschlüssel liegen keine Daten zur Stimmen-Verteilung vor
# fehlende Werte mit denen des am nächsten verfügbaren AGS 8-stellig ersetzen
marktstd_test = marktstd_test.sort_values(by='AGS-8')
marktstd_filled = marktstd_test.fillna(method='ffill').fillna(method='bfill')
#marktstd_filled.isna().sum()
# Quelle: ChatGPT (siehe ChatGPT Protokoll, Seite 4)

In [174]:
# Tabelle Bundestagswahlen 
selected_columns = ['AGS-8'] + list(marktstd_test.columns[-12:])
wahlen = marktstd_filled[selected_columns].reset_index(drop=True)
wahlen

,AGS-8,CDU_Prozent,SPD_Prozent,AfD_Prozent,FDP_Prozent,DIE LINKE_Prozent,GRÜNE_Prozent,CSU_Prozent,FREIE WÄHLER_Prozent,Die PARTEI_Prozent,Tierschutzpartei_Prozent,NPD_Prozent,PIRATEN_Prozent
0,01001000,15.63,20.63,5.28,6.40,5.62,35.58,0.00,0.89,0.00,0.00,0.00,0.00
1,01002000,17.26,29.63,4.70,7.37,4.95,28.71,0.00,0.72,1.98,0.00,0.00,0.00
2,01003000,21.27,34.40,6.43,7.11,3.16,22.05,0.00,1.33,1.70,0.00,0.00,0.00
3,01004000,25.26,35.03,9.31,9.40,2.88,13.14,0.00,1.40,1.77,0.00,0.00,0.00
4,01051001,31.53,27.38,8.43,14.79,3.95,9.91,0.00,2.21,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11102,16077048,26.56,19.11,28.97,10.46,5.03,3.22,0.00,0.00,0.00,0.00,0.00,0.00
11103,16077049,27.07,12.71,29.28,12.71,8.29,5.52,0.00,0.00,0.00,0.00,0.00,0.00
11104,16077051,27.07,12.71,29.28,12.71,8.29,5.52,0.00,0.00,0.00,0.00,0.00,0.00
11105,16077052,23.00,17.18,30.49,8.79,7.49,4.26,0.00,0.00,0.00,0.00,0.00,0.00


In [175]:
#wahlen.isna().sum()

# SQLite Datenbank aufbauen

In [176]:
# SQLite Datenbank Pfad 
SQLITE_FILE_PATH = os.path.join("..", "output", "dwh.sqlite3")

In [177]:
# Connection zur Datenbank 
con = sqlite3.connect(SQLITE_FILE_PATH)

## Tabellen exportieren

In [178]:
# Marktstammdaten-Tabelle mit der neuen, bereinigten Tabelle überschreiben
marktstd.to_sql("marktstammdaten", con=con, if_exists="replace", index=False, dtype={
    'DatumLetzteAktualisierung': 'DATE',
    'Registrierungsdatum': 'DATE',
    'Inbetriebnahmedatum': 'DATE'
})

3173010

In [179]:
# Bevölkerungsdichte-Tabelle anlegen
bev_dichte.to_sql("bevoelkerungsdichte", con=con, if_exists="replace", index=False)

400

In [180]:
# Bevölkerungsentwicklung-Tabelle anlegen
bev_entw.to_sql("bevoelkerungsentwicklung", con=con, if_exists="replace", index=False)

11107

In [181]:
# Eigenheimquote-Tabelle anlegen
eigenheimquote.to_sql("eigenheimquote", con=con, if_exists="replace", index=False)

11107

In [182]:
# Gini_Index-Tabelle anlegen
gini_raw.to_sql("gini_index", con=con, if_exists="replace", index=False)

19

In [183]:
# Einkommensentwicklung-Tabelle anlegen
einkommen.to_sql("einkommensentwicklung", con=con, if_exists="replace", index=False)

445

In [184]:
# Bruttoinlandsprodukt-Tabelle anlegen
bip_raw.to_sql("bip", con=con, if_exists="replace", index=False)

401

In [185]:
# Arbeitslosenquote-Tabelle anlegen
arbeitslos_raw.to_sql("arbeitslosenquote", con=con, if_exists="replace", index=False)

16

In [186]:
# Altersstruktur-Tabelle anlegen
alter.to_sql("altersstruktur", con=con, if_exists="replace", index=False)

11107

In [187]:
# Migrationshintergrund-Tabelle anlegen
migration.to_sql("migrationshintergrund", con=con, if_exists="replace", index=False)

11107

In [188]:
# Familienstand-Tabelle anlegen
fam_stand.to_sql("familienstand", con=con, if_exists="replace", index=False)

11107

In [189]:
# Bildung-Tabelle anlegen
bildung_raw.to_sql("bildung", con=con, if_exists="replace", index=False)

400

In [190]:
# Ausländeranteil-Tabelle anlegen
auslaender_raw.to_sql("auslaenderanteil", con=con, if_exists="replace", index=False)

400

In [191]:
# Kriminalität-Tabelle anlegen
krimi_raw.to_sql("kriminalitaet", con=con, if_exists="replace", index=False)

401

In [192]:
# Bundestagswahlergenisse-Tabelle anlegen
wahlen.to_sql("bundestagswahlergenisse", con=con, if_exists="replace", index=False)

11107

## Joins

In [193]:
# Join marktstammdaten <> bevoelkerungsdichte
query_01 = """  
SELECT *
FROM marktstammdaten 
LEFT JOIN bevoelkerungsdichte
ON marktstammdaten.[AGS-5] = bevoelkerungsdichte.[AGS-5]
LIMIT 5
"""
#bevoelkerung = pd.read_sql_query(query_01, con=con)
#bevoelkerung

In [194]:
# Join marktstammdaten <> bevoelkerungsentwicklung
query_02 = """  
SELECT *
FROM marktstammdaten 
LEFT JOIN bevoelkerungsentwicklung
ON marktstammdaten.[AGS-8] = bevoelkerungsentwicklung.[AGS-8]
LIMIT 5
"""
#bevoelkerungsentw = pd.read_sql_query(query_02, con=con)
#bevoelkerungsentw

In [195]:
# Join marktstammdaten <> eigenheimquote
query_03 = """  
SELECT *
FROM marktstammdaten 
LEFT JOIN eigenheimquote
ON marktstammdaten.[AGS-8] = eigenheimquote.[AGS-8]
LIMIT 5
"""
#eigenheim = pd.read_sql_query(query_03, con=con)
#eigenheim

In [196]:
# Join marktstammdaten <> gini_index
query_04 = """  
SELECT *
FROM marktstammdaten 
LEFT JOIN gini_index
ON marktstammdaten.Bundesland = gini_index.Bundesland
LIMIT 5
"""
#gini = pd.read_sql_query(query_04, con=con)
#gini

In [197]:
# Join marktstammdaten <> einkommensentwicklung
query_05 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN einkommensentwicklung
ON marktstammdaten.[AGS-5] = einkommensentwicklung.[AGS-5]
LIMIT 5
"""
#einkommensentw = pd.read_sql_query(query_05, con=con)
#einkommensentw

In [198]:
# Join marktstammdaten <> bip
query_06 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN bip
ON marktstammdaten.[AGS-5] = bip.[AGS-5]
LIMIT 5
"""
#bip = pd.read_sql_query(query_06, con=con)
#bip

In [199]:
# Join marktstammdaten <> arbeitslosenquote
query_07 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN arbeitslosenquote
ON marktstammdaten.Bundesland = arbeitslosenquote.Bundesland
LIMIT 5
"""
#arbeitslquote = pd.read_sql_query(query_07, con=con)
#arbeitslquote

In [200]:
# Join marktstammdaten <> altersstruktur
query_08 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN altersstruktur
ON marktstammdaten.[AGS-8]= altersstruktur.[AGS-8]
LIMIT 5
"""
#alter = pd.read_sql_query(query_08, con=con)
#alter

In [201]:
# Join marktstammdaten <> migrationshintergrund
query_09 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN migrationshintergrund
ON marktstammdaten.[AGS-8]= migrationshintergrund.[AGS-8]
LIMIT 5
"""
#migration = pd.read_sql_query(query_09, con=con)
#migration

In [202]:
# Join marktstammdaten <> familienstand
query_10 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN familienstand
ON marktstammdaten.[AGS-8]= familienstand.[AGS-8]
LIMIT 5
"""
#familienst = pd.read_sql_query(query_10, con=con)
#familienst

In [203]:
# Join marktstammdaten <> bildung
query_11 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN bildung
ON marktstammdaten.[AGS-5]= bildung.[AGS-5]
LIMIT 5
"""
#bildung = pd.read_sql_query(query_11, con=con)
#bildung

In [204]:
# Join marktstammdaten <> auslaenderanteil
query_12 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN auslaenderanteil
ON marktstammdaten.[AGS-5]= auslaenderanteil.[AGS-5]
LIMIT 5
"""
#auslaenderanteil = pd.read_sql_query(query_12, con=con)
#auslaenderanteil

In [205]:
# Join marktstammdaten <> kriminalitaet
query_13 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN kriminalitaet
ON marktstammdaten.[AGS-5]= kriminalitaet.[AGS-5]
LIMIT 5
"""
#riminalitaet = pd.read_sql_query(query_13, con=con)
#kriminalitaet

In [206]:
# Join marktstammdaten <> bundestagswahlergenisse
query_14 = """
SELECT *
FROM marktstammdaten 
LEFT JOIN bundestagswahlergenisse
ON marktstammdaten.[AGS-8]= bundestagswahlergenisse.[AGS-8]
LIMIT 5
"""
#bundestagswahlen = pd.read_sql_query(query_14, con=con)
#bundestagswahlen

In [207]:
# Verbindung zur Datenbank trennen
con.close()